In [3]:
import numpy as np
import ast
import os
import re
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.sparse import eye, kron, identity, csr_matrix, csc_matrix, lil_matrix, dok_matrix, issparse, coo_matrix
from scipy.sparse.linalg import eigsh, eigs, lobpcg, LinearOperator, ArpackNoConvergence
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
import itertools
from itertools import product
from functools import reduce
import torch
import torch.optim as optim
from torch.autograd import Variable
import sympy as sp
from collections import Counter, defaultdict, deque
from IPython.display import display, HTML
import networkx as nx
from sympy.combinatorics import Permutation, PermutationGroup

In [4]:
bonds = [
    (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
    (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
    (2, 6), (2, 7), (2, 8), (2, 9), (3, 4),
    (3, 5), (3, 10), (3, 11), (4, 5), (4, 8),
    (4, 10), (5, 9), (5, 11), (6, 7), (6, 8),
    (6, 10), (7, 9), (7, 11), (8, 10), (9, 11)
]
G = nx.Graph()
G.add_nodes_from(range(12))
G.add_edges_from(bonds)

# --- all automorphisms (size 120)
GM = nx.algorithms.isomorphism.GraphMatcher(G, G)
autos = sorted({tuple(iso[i] for i in range(12)) for iso in GM.isomorphisms_iter()})
assert len(autos) == 120, f"Expected 120 automorphisms, got {len(autos)}"

# --- planar embedding to get a rotation system
is_planar, embedding = nx.check_planarity(G, True)
assert is_planar

def neighbors_cw(v):
    # returns the neighbors of v in clockwise order per the embedding
    return list(embedding.neighbors_cw_order(v))

def is_cyclic_shift(a, b):
    # checks if list a is a cyclic rotation of list b (same orientation)
    if len(a) != len(b): return False
    if not a: return True
    try:
        i = b.index(a[0])
    except ValueError:
        return False
    return all(a[k] == b[(i + k) % len(b)] for k in range(len(a)))

def preserves_rotation_system(p):
    # For each vertex v, the image of its CW neighbor list must be a CW cyclic shift
    for v in range(12):
        src_cw = neighbors_cw(v)
        # map neighbors under p
        mapped = [p[n] for n in src_cw]
        # target cw list at p[v]
        tgt_cw = neighbors_cw(p[v])
        if not is_cyclic_shift(mapped, tgt_cw):
            return False
    return True

# --- keep only orientation-preserving automorphisms (the 60 rotations)
rotations = [p for p in autos if preserves_rotation_system(p)]
assert len(rotations) == 60, f"Expected 60 rotations, got {len(rotations)}"

# --- inversion: map each vertex to the unique vertex at graph distance 3
inv = [-1]*12
for v in range(12):
    dist = nx.single_source_shortest_path_length(G, v)
    opp = [u for u, d in dist.items() if d == 3]
    assert len(opp) == 1
    inv[v] = opp[0]
inv = tuple(inv)
assert all(inv[inv[i]] == i for i in range(12))

def compose(p, q):
    # (p ∘ q)[i] = p[q[i]]
    return tuple(p[q[i]] for i in range(12))

# --- full I_h = rotations ∪ (inversion ∘ rotations)
I_h = {tuple(r) for r in rotations}
I_h.update(compose(inv, r) for r in rotations)
I_h = sorted(I_h)
assert len(I_h) == 120, f"Expected 120 elements, got {len(I_h)}"

print("Rotations:", len(rotations))
print("Full I_h:", len(I_h))
print(I_h)

Rotations: 60
Full I_h: 120
[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 2, 3, 5, 4, 7, 6, 9, 8, 11, 10), (0, 1, 4, 7, 2, 9, 10, 3, 8, 5, 6, 11), (0, 1, 4, 7, 9, 2, 3, 10, 5, 8, 11, 6), (0, 1, 5, 6, 2, 8, 11, 3, 9, 4, 7, 10), (0, 1, 5, 6, 8, 2, 3, 11, 4, 9, 10, 7), (0, 1, 8, 11, 5, 9, 10, 6, 4, 2, 3, 7), (0, 1, 8, 11, 9, 5, 6, 10, 2, 4, 7, 3), (0, 1, 9, 10, 4, 8, 11, 7, 5, 2, 3, 6), (0, 1, 9, 10, 8, 4, 7, 11, 2, 5, 6, 3), (1, 0, 3, 2, 6, 7, 4, 5, 10, 11, 8, 9), (1, 0, 3, 2, 7, 6, 5, 4, 11, 10, 9, 8), (1, 0, 6, 5, 3, 11, 8, 2, 10, 7, 4, 9), (1, 0, 6, 5, 11, 3, 2, 8, 7, 10, 9, 4), (1, 0, 7, 4, 3, 10, 9, 2, 11, 6, 5, 8), (1, 0, 7, 4, 10, 3, 2, 9, 6, 11, 8, 5), (1, 0, 10, 9, 7, 11, 8, 4, 6, 3, 2, 5), (1, 0, 10, 9, 11, 7, 4, 8, 3, 6, 5, 2), (1, 0, 11, 8, 6, 10, 9, 5, 7, 3, 2, 4), (1, 0, 11, 8, 10, 6, 5, 9, 3, 7, 4, 2), (2, 3, 0, 1, 6, 7, 4, 5, 8, 9, 10, 11), (2, 3, 0, 1, 7, 6, 5, 4, 9, 8, 11, 10), (2, 3, 6, 5, 0, 9, 10, 1, 8, 7, 4, 11), (2, 3, 6, 5, 9, 0, 1, 10, 7, 8, 11, 4), (2, 3, 7, 4

In [11]:
# size of group
G_size = 120
d_Hg = 5

# characters of H_g by class (ordered as above)
chi_Hg = [5, 1, -1, 0, 0, 5, 1, -1, 0, 0]

# number of elements in each class
class_sizes = [1, 15, 20, 12, 12, 1, 15, 20, 12, 12]

# function that assigns each permutation g to class index 0..9
# (you must implement this, e.g., by checking rotation order or parity)
def classify_element(perm):
    
    #Classify a group element (as a tuple/list of 12 ints) into one of the 10 conjugacy classes of Ih.
    #Returns an integer 0..9 (class index).
    
    # Identity
    if tuple(perm) == tuple(range(12)):
        return 0

    # Inversion
    if tuple(perm) == inv:
        return 5

    # Is it a pure rotation?
    is_rotation = perm in rotations

    # Compose with inversion to check if it's inversion * rotation
    is_inverted = False
    if not is_rotation:
        for r in rotations:
            if tuple(perm) == tuple(inv[r[i]] for i in range(12)):
                is_inverted = True
                underlying_rot = r
                break

    # Helper: get order and cycle structure
    def perm_cycles(p):
        seen = [False]*12
        cycles = []
        for i in range(12):
            if not seen[i]:
                j = i
                cycle = []
                while not seen[j]:
                    seen[j] = True
                    cycle.append(j)
                    j = p[j]
                if len(cycle) > 1:
                    cycles.append(tuple(cycle))
        return cycles

    def perm_order(p):
        from math import lcm
        cycles = perm_cycles(p)
        order = 1
        for cyc in cycles:
            order = lcm(order, len(cyc))
        return order

    # Classify by order and type
    if is_rotation:
        order = perm_order(perm)
        cycles = perm_cycles(perm)
        if order == 2:
            return 1
        elif order == 3:
            return 2
        elif order == 5:
            # Two types: distinguished by which vertices are fixed
            fixed = [i for i in range(12) if perm[i] == i]
            # Each 5-fold rotation fixes 2 points
            # The set of fixed points distinguishes the type
            # Let's sort and use the tuple as a key
            # Build a list of all such fixed point pairs for all 5-fold rotations
            # The first 12 unique pairs are type 1, the next 12 are type 2
            # So, build a sorted list of all fixed pairs
            all_5folds = [p for p in rotations if perm_order(p) == 5]
            fixed_pairs = sorted([tuple(sorted([i for i in range(12) if p[i] == i])) for p in all_5folds])
            my_pair = tuple(sorted(fixed))
            idx = fixed_pairs.index(my_pair)
            return 3 if idx < 12 else 4
    elif is_inverted:
        order = perm_order(underlying_rot)
        cycles = perm_cycles(underlying_rot)
        if order == 2:
            return 6
        elif order == 3:
            return 7
        elif order == 5:
            fixed = [i for i in range(12) if underlying_rot[i] == i]
            all_5folds = [p for p in rotations if perm_order(p) == 5]
            fixed_pairs = sorted([tuple(sorted([i for i in range(12) if p[i] == i])) for p in all_5folds])
            my_pair = tuple(sorted(fixed))
            idx = fixed_pairs.index(my_pair)
            return 8 if idx < 12 else 9

    raise ValueError("Permutation does not match any known class.")


# your D(g): representation on Hilbert space
def D(perm):
    # for example, permutation matrix acting on 12 sites
    mat = np.zeros((12,12))
    for i in range(12):
        mat[perm[i], i] = 1.0
  
    return mat  

# --- build projector
P_Hg = np.zeros((12,12))
for g in I_h:
    print("Classifying element:", g)
    class_idx = classify_element(g)
    print("Class index:", class_idx)
    print("Character:", chi_Hg[class_idx])
    P_Hg += np.conjugate(chi_Hg[class_idx]) * D(g)

P_Hg *= d_Hg / G_size

print("||P_Hg^2 - P_Hg|| =", np.linalg.norm(P_Hg @ P_Hg - P_Hg))
print("||P_Hg - P_Hg†|| =", np.linalg.norm(P_Hg - P_Hg.conj().T))
print("Tr(P_Hg) =", np.trace(P_Hg))

Classifying element: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
Class index: 0
Character: 5
Classifying element: (0, 1, 2, 3, 5, 4, 7, 6, 9, 8, 11, 10)
Class index: 6
Character: 1
Classifying element: (0, 1, 4, 7, 2, 9, 10, 3, 8, 5, 6, 11)
Class index: 6
Character: 1
Classifying element: (0, 1, 4, 7, 9, 2, 3, 10, 5, 8, 11, 6)
Class index: 3
Character: 0
Classifying element: (0, 1, 5, 6, 2, 8, 11, 3, 9, 4, 7, 10)
Class index: 3
Character: 0
Classifying element: (0, 1, 5, 6, 8, 2, 3, 11, 4, 9, 10, 7)
Class index: 6
Character: 1
Classifying element: (0, 1, 8, 11, 5, 9, 10, 6, 4, 2, 3, 7)
Class index: 3
Character: 0
Classifying element: (0, 1, 8, 11, 9, 5, 6, 10, 2, 4, 7, 3)
Class index: 6
Character: 1
Classifying element: (0, 1, 9, 10, 4, 8, 11, 7, 5, 2, 3, 6)
Class index: 6
Character: 1
Classifying element: (0, 1, 9, 10, 8, 4, 7, 11, 2, 5, 6, 3)
Class index: 3
Character: 0
Classifying element: (1, 0, 3, 2, 6, 7, 4, 5, 10, 11, 8, 9)
Class index: 1
Character: 1
Classifying element: (1, 0, 3, 2

In [10]:
# Human-readable class names (optional)
class_labels = [
    "Identity",
    "C2 (2-fold rotation)",
    "C3 (3-fold rotation)",
    "C5 (5-fold rotation, type 1)",
    "C5 (5-fold rotation, type 2)",
    "Inversion",
    "C2*inv",
    "C3*inv",
    "C5*inv (type 1)",
    "C5*inv (type 2)"
]

print("Element index | Permutation | Class index | Class label")
for idx, g in enumerate(I_h):
    class_idx = classify_element(g)
    print(f"{idx:3d} | {g} | {class_idx} | {class_labels[class_idx]}")

class_indices = [classify_element(g) for g in I_h]
counts = Counter(class_indices)
for idx, count in sorted(counts.items()):
    print(f"Class {idx} ({class_labels[idx]}): {count} elements")

Element index | Permutation | Class index | Class label
  0 | (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11) | 0 | Identity
  1 | (0, 1, 2, 3, 5, 4, 7, 6, 9, 8, 11, 10) | 6 | C2*inv
  2 | (0, 1, 4, 7, 2, 9, 10, 3, 8, 5, 6, 11) | 6 | C2*inv
  3 | (0, 1, 4, 7, 9, 2, 3, 10, 5, 8, 11, 6) | 3 | C5 (5-fold rotation, type 1)
  4 | (0, 1, 5, 6, 2, 8, 11, 3, 9, 4, 7, 10) | 3 | C5 (5-fold rotation, type 1)
  5 | (0, 1, 5, 6, 8, 2, 3, 11, 4, 9, 10, 7) | 6 | C2*inv
  6 | (0, 1, 8, 11, 5, 9, 10, 6, 4, 2, 3, 7) | 3 | C5 (5-fold rotation, type 1)
  7 | (0, 1, 8, 11, 9, 5, 6, 10, 2, 4, 7, 3) | 6 | C2*inv
  8 | (0, 1, 9, 10, 4, 8, 11, 7, 5, 2, 3, 6) | 6 | C2*inv
  9 | (0, 1, 9, 10, 8, 4, 7, 11, 2, 5, 6, 3) | 3 | C5 (5-fold rotation, type 1)
 10 | (1, 0, 3, 2, 6, 7, 4, 5, 10, 11, 8, 9) | 1 | C2 (2-fold rotation)
 11 | (1, 0, 3, 2, 7, 6, 5, 4, 11, 10, 9, 8) | 5 | Inversion
 12 | (1, 0, 6, 5, 3, 11, 8, 2, 10, 7, 4, 9) | 8 | C5*inv (type 1)
 13 | (1, 0, 6, 5, 11, 3, 2, 8, 7, 10, 9, 4) | 1 | C2 (2-fold rotation)
 1

In [ ]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def icosahedral_bonds():
    
    bonds = [
        (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
        (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
        (2, 6), (2, 7), (2, 8), (2, 9), (3, 4),
        (3, 5), (3, 10), (3, 11), (4, 5), (4, 8),
        (4, 10), (5, 9), (5, 11), (6, 7), (6, 8),
        (6, 10), (7, 9), (7, 11), (8, 10), (9, 11)
    ]
    return bonds

def transverse_field_ising_icosahedral(N, J, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def ising_icosahedron(N, J):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure without transverse field.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    return H

def transverse_field_icosahedral(N, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

#######################################################################################################################

'''
def partial_trace_qubit(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

def partial_trace_qubit_torch(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits using PyTorch."""
    keep_dims = torch.prod(torch.tensor([dims[i] for i in keep]))
    trace_dims = torch.prod(torch.tensor([dims[i] for i in range(len(dims)) if i not in keep]))
    rho = rho.view(keep_dims, trace_dims, keep_dims, trace_dims)
    # Compute the partial trace
    traced_rho = torch.zeros((keep_dims, keep_dims), dtype=rho.dtype)
    for i in range(trace_dims):
        traced_rho += rho[:, i, :, i]
    #return traced_rho.view(keep_dims, keep_dims)
    return traced_rho'''

def isket_numpy(arr):
    """
    Check if a NumPy array is a ket (column vector).

    Parameters:
    - arr: np.ndarray, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a NumPy array")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_numpy(Q, sel, dims): # numpy function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using NumPy.

    Parameters:
    - Q: numpy object, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - numpy object, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = np.asarray(dims[0], dtype=np.int32).ravel()
    nd = len(rd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = np.array([sel])
    else:
        sel = np.asarray(sel)
    sel = list(np.sort(sel))
    
    # Dimensions of the subsystems to keep
    dkeep = (rd[sel]).tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(np.arange(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = (rd[qtrace]).tolist()
    
    # Reshape the density matrix or state vector
    rd = list(rd)
    if isket_numpy(Q):
        # Reshape and transpose for state vector
        vmat = (Q
                .reshape(rd)
                .transpose(sel + qtrace)
                .reshape([np.prod(dkeep), np.prod(dtrace)]))
        # Compute the reduced density matrix
        rhomat = vmat.dot(vmat.conj().T)
    else:
        # Reshape and transpose for density matrix
        rhomat = np.trace(Q
                          .reshape(rd + rd)
                          .transpose(qtrace + [nd + q for q in qtrace] +
                                     sel + [nd + q for q in sel])
                          .reshape([np.prod(dtrace),
                                    np.prod(dtrace),
                                    np.prod(dkeep),
                                    np.prod(dkeep)]))
    return rhomat


def ptrace_sparse(psi_sparse, keep, dims):
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.

    Args:
        psi_sparse (scipy.sparse matrix): Full density matrix of shape (D, D), where D = product(dims)
        keep (list of int): Subsystems to keep (indices, 0-indexed)
        dims (list of int): List of subsystem dimensions, e.g., [2]*n for n qubits

    Returns:
        scipy.sparse.csr_matrix: Reduced density matrix over kept subsystems
    """
    if not issparse(psi_sparse):
        raise ValueError("psi_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if psi_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    # Prepare output
    data = []
    row_idx = []
    col_idx = []

    # Precompute bit masks
    def idx_to_bits(idx):
        return np.array(list(np.binary_repr(idx, width=n))).astype(int)
    

    psi_sparse = psi_sparse.tocoo()
    for i, j, val in zip(psi_sparse.row, psi_sparse.col, psi_sparse.data):
        bi = idx_to_bits(i)
        bj = idx_to_bits(j)


        # Only sum terms where traced-out subsystems agree
        if np.all(bi[trace] == bj[trace]):
            # Extract kept bits and convert to reduced indices
            #print('condition met for i, j:', i, j)
            i_red_bits = bi[keep]
            j_red_bits = bj[keep]
            i_red = int("".join(i_red_bits.astype(str)), 2)
            j_red = int("".join(j_red_bits.astype(str)), 2)


            data.append(val)
            row_idx.append(i_red)
            col_idx.append(j_red)
    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()


def isket_torch(arr):
    """
    Check if a PyTorch tensor is a ket (column vector).

    Parameters:
    - arr: torch.Tensor, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, torch.Tensor):
        raise ValueError("Input must be a PyTorch tensor")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_torch(Q, sel, dims): # torch function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using PyTorch.

    Parameters:
    - Q: torch.Tensor, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - torch.Tensor, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = torch.tensor(dims[0], dtype=torch.int32).flatten()
    nd = len(rd)
    #print("rd", rd)
    #print("nd", nd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = torch.tensor([sel])
    else:
        sel = torch.tensor(sel)
    sel = torch.sort(sel).values.tolist()
    
    # Dimensions of the subsystems to keep
    dkeep = rd[sel].tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(range(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = rd[qtrace].tolist()
    
    # Reshape the density matrix or state vector
    rd = rd.tolist()
    if isket_torch(Q):
        # Reshape and transpose for state vector
        reshaped_Q = Q.reshape(rd)
        #print(reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(sel + qtrace)
        #print(transposed_Q.shape)
        vmat = transposed_Q.reshape([torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dtrace))])
        #print(vmat.shape)
        # Compute the reduced density matrix
        rhomat = vmat @ vmat.conj().T
        #print(rhomat.shape)
    else:
        # Reshape and transpose for density matrix
        reshaped_Q = Q.reshape(rd + rd)
        #print("reshaped_Q", reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(qtrace + [nd + q for q in qtrace] + sel + [nd + q for q in sel])
        #print("transposed_Q", transposed_Q.shape)
        reshaped_transposed_Q = transposed_Q.reshape([torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))])
        #print("reshaped_transposed_Q", reshaped_transposed_Q.shape)
        #rhomat = torch.trace(reshaped_transposed_Q)
        rhomat = torch.einsum('iikl->kl', reshaped_transposed_Q)
        # Trace out the first two dimensions
        #rhomat = torch.zeros((torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))), dtype=Q.dtype)
        #for i in range(reshaped_transposed_Q.shape[0]):
        #    for j in range(reshaped_transposed_Q.shape[1]):
        #        rhomat += reshaped_transposed_Q[i, j, :, :]
        #print("rhomat", rhomat.shape)
    return rhomat

def entanglement_entropy(psi, subsystem, total_size):

    '''Computes the bipartite entanglement entropy of a pure state.
    
    Parameters:
    psi : np.array
        The wavefunction (state vector) of the full system.
    subsystem_size : int
        The number of qubits in subsystem A.
    total_size : int
        The total number of qubits in the system.
    
    Returns:
    float
        The von Neumann entanglement entropy S_A.'''
    
    psi_matrix =  np.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_numpy(psi_matrix, subsystem, [[2]*total_size, [2]*total_size])  # Partial trace over B
    
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rho_A)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch(psi, subsystem, total_size):
    """
    Computes the bipartite entanglement entropy of a pure state using PyTorch.

    Parameters:
    - psi: torch.Tensor (complex), the wavefunction (state vector) of the full system.
    - subsystem_size: int, the number of qubits in subsystem A.
    - total_size: int, the total number of qubits in the system.

    Returns:
    - torch.Tensor (scalar), the von Neumann entanglement entropy S_A.
    """

    if not isinstance(psi, torch.Tensor):
        psi = torch.tensor(psi, dtype=torch.complex64)
    
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Compute the density matrix |psi><psi|
    psi_matrix = torch.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_torch(psi_matrix, subsystem, [[2] * total_size, [2] * total_size])  # Partial trace over B

    #rho_A = rho_A.to(dtype=torch.float64)
    
    # Compute eigenvalues of rho_A
    eigvals = torch.linalg.eigvalsh(rho_A)

    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigvals = eigvals[eigvals > 0]

    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -torch.sum(eigvals * torch.log2(eigvals))

    return entropy

def entanglement_entropy_qutip(psi, subsystem, total_size):
    
    # Convert the wavefunction to a QuTiP Qobj
    density_matrix = np.outer(psi, psi.conj())
    density_matrix_qobj = Qobj(density_matrix, dims=[[2]*total_size, [2]*total_size])

    rho_A = ptrace(density_matrix_qobj, subsystem)
    # Compute the von Neumann entropy S_A
    entropy = entropy_vn(rho_A, base=2)
    
    return entropy

def entanglement_entropy_np_ptrace(rdm):
    # rdm already computed and converted to numpy
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rdm)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch_ptrace(rdm):

    eigvals = torch.linalg.eigvalsh(rdm)
    eigvals = eigvals[eigvals > 0]
    entropy = -torch.sum(eigvals * torch.log2(eigvals))
    return entropy


def entanglement_entropy_qutip_torch(psi, N):
    """
    Compute the von Neumann entanglement entropy using qutip.

    Parameters:
    - psi: torch.Tensor (complex), state vector of a quantum system.
    - N: int, total number of qubits.

    Returns:
    - torch.Tensor (scalar), von Neumann entropy.
    """
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Convert PyTorch tensor to NumPy for QuTiP
    psi_np = psi.detach().numpy()

    rho_np = np.outer(psi_np, psi_np.conj())
    rho_qobj = Qobj(rho_np, dims=[[2] * N, [2] * N])

    rho_A = ptrace(rho_qobj, list(range(N // 2)))

    # Compute von Neumann entropy
    entropy = entropy_vn(rho_A, base=2)  # Compute in log base 2

    # Convert back to PyTorch tensor to allow gradient flow
    return torch.tensor(entropy, dtype=torch.float32, requires_grad=True)

#######################################################################################################################

# Define the linear combination function - numpy
def linear_combination_np(coeffs, psis):
    # Ensure psis are numpy tensors
    psi_np = [np.array(psi) for psi in psis]
    # Compute the linear combination in PyTorch
    psi = sum(c * psi for c, psi in zip(coeffs, psis))
    
    return psi

# Define the linear combination function - torch
def linear_combination(coeffs, psis):
    # Ensure psis are PyTorch tensors
    psis_torch = [torch.tensor(psi, dtype=torch.complex64) if not isinstance(psi, torch.Tensor) else psi for psi in psis]
    
    # Compute the linear combination in PyTorch
    psi_torch = sum(c * psi for c, psi in zip(coeffs, psis_torch))
    
    return psi_torch

# Define the linear combination function - torch but after computing the ptrace of outer products of scars
def linear_combination_outer(coeffs, outs):
    # Ensure outs are PyTorch tensors
    outs_torch = [torch.tensor(out, dtype=torch.complex64) if not isinstance(out, torch.Tensor) else out for out in outs]
    torch_coeffs = torch.tensor(coeffs, dtype=torch.complex64)

    # Compute the PyTorch tensor of out_coeffs which is the product of all possible combinations of c_i^* times c_j
    out_coeffs = torch.zeros((len(torch_coeffs), len(torch_coeffs)), dtype=torch.complex64)
    for i in range(len(torch_coeffs)):
        for j in range(len(torch_coeffs)):
            out_coeffs[i, j] = torch.conj(torch_coeffs[i]) * torch_coeffs[j]
    
    # Compute the linear combination in PyTorch
    lin_torch = sum(out_coeffs[i, j] * outs_torch[i] for i in range(len(coeffs)) for j in range(len(coeffs)))
    
    return lin_torch

######################################################

# Function to apply permutation to a given spin configuration
def apply_permutation(state_bits, N, perm):
    new_bits = [0] * N
    for i in range(N):
        new_bits[perm[i]] = state_bits[i]  # Map value at index i to perm[i]
    return new_bits

# Function to convert an index to its binary representation
def index_to_binary(index, num_qubits):
    return format(index, f'0{num_qubits}b')

In [ ]:
N = 12  # Number of spins
J = 1.0  # Interaction strength
h = 3.0  # Transverse field strength # this is the value in the paper. maybe try  other values too, including the critical value one (h=J=1)

#keep_qubits = [0, 4, 5]
#keep_qubits = [0, 4, 5, 11] # 4 spins rdm full rank
#keep_qubits = [0, 4, 5, 9]  # 4 spins - 2 adjacent triangular plaquettes - this is when i observe exactly 5 degenerate scars
#keep_qubits = [0, 4, 5, 9, 11]  # 5 spins - 3 adjacent triangular plaquettes
keep_qubits = [2, 4, 5, 8, 9] # 5 spins - pentagon around 0

trace_qubits = [i for i in range(N) if i not in keep_qubits]


# Assuming transverse_field_ising is defined and returns a sparse Hermitian matrix
H = transverse_field_ising_icosahedral(N, J, h)

# project H into the even-parity, H_g block:
# (You can then run eigsh on the smaller projected space by building an orthonormal basis for the range.)
H_Hg_full = (P @ (H @ P)).tocsr()

#print(f"Hamiltonian shape: {H.shape}")
#print(f"Non-zero elements in H: {H.nnz}")

# Define the C5 permutation mapping for your case - around axis through vert 9 and 10
perm = {
    0: 2,
    2: 7,
    7: 11,
    11: 5,
    5: 0,
    1: 3,
    3: 4,
    4: 8,
    8: 6,
    6: 1,
    9: 9,   # Fixed
    10: 10  # Fixed
}

In [ ]:
# All eigenvalues of H
# Compute all eigenvalues and corresponding eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(H_even_Hg.toarray())

print(f"Eigenvalues of H: {np.sort(eigenvalues)}")

# Count the number of (near-)zero components in each eigenvector
zero_threshold = 1e-6
nzero_counts = []
for i in range(eigenvectors.shape[1]):
    num_nzero = np.sum(np.abs(eigenvectors[:, i]) > zero_threshold)
    nzero_counts.append(num_nzero)
    print(f"Eigenvector {i}, eigenvalue {eigenvalues[i]}: {num_nzero} components > {zero_threshold}")

# Optionally, print a summary
print("Zero count distribution:", Counter(nzero_counts))

# Check if each eigenvector is complex or real
for i in range(eigenvectors.shape[1]):
    vec = eigenvectors[:, i]
    #if np.any(np.abs(vec.imag) > 1e-14):
    #    print(f"Eigenvector {i} is complex.")
    #else:
    #    print(f"Eigenvector {i} is real.")

In [ ]:
sparse_indices = []
for i in range(eigenvectors.shape[1]):
    if nzero_counts[i] < 1000:
        print(f"Eigenvector {i} with eigenvalue {eigenvalues[i]} has {nzero_counts[i]} non-zero components.")
        sparse_indices.append(i)

print("Indices of sparse eigenvectors:", sparse_indices)

plt.plot(eigenvalues, nzero_counts, 'o')
plt.show()

In [ ]:
#sparse_indices = [1266,1267,1268,1269,1270, 1527] # 5 scars for 5 spins rdm with keep_qubits = [2,4,5,8,9] and h=3J   

In [ ]:
# scars + tf field - sparse


for i in tqdm(sparse_indices):
    # Construct the density matrix for each eigenvector (as sparse)
    min_eigenvector = eigenvectors[:, i]
    print("Nonzero elements in state vector (tol=1e-12):", np.sum(np.abs(min_eigenvector) > 1e-6))
    min_eigenvector[np.abs(min_eigenvector) < 1e-7] = 0
    min_eig_sparse = csr_matrix(min_eigenvector.reshape(-1, 1))  # Convert to sparse column vector
    
    # Apply the transverse field operator to the sparse vector
    vecs = transverse_field_icosahedral(N, h) @ min_eig_sparse
    vecs = vecs.toarray()  # Convert to dense array for further processing
    
    # Print information about the result
    print(f"Shape of result: {vecs.shape}")
    print(f"for {i}-th eigenvector: Number of nonzero elements in result: {np.count_nonzero(vecs)}")
    print(f"Max absolute value in result: {np.max(np.abs(vecs.data))}")

In [ ]:
# Check if commutator [Htf, Hi] is zero on eigenvectors of sparse indices
print("Checking commutator [Htf, Hi] on sparse eigenvectors...")

# Build the Hamiltonians
Hi = ising_icosahedron(N, J)
Htf = transverse_field_icosahedral(N, h)

# Compute the commutator [Htf, Hi] = Htf * Hi - Hi * Htf
print("Computing commutator [Htf, Hi]...")
commutator = Htf @ Hi - Hi @ Htf
print(f"Commutator computed. Shape: {commutator.shape}")
print(f"Commutator nnz: {commutator.nnz}")

# Check the action of the commutator on each sparse eigenvector
commutator_norms = []
commutator_results = []

for i in sparse_indices:
    vec = eigenvectors[:, i]
    eigenval = eigenvalues[i]
    
    print(f"\nTesting eigenvector {i} (eigenvalue: {eigenval:.6f})...")
    
    # Apply commutator to the eigenvector
    comm_vec = commutator @ vec
    
    # Compute the norm of the result
    comm_norm = np.linalg.norm(comm_vec)
    commutator_norms.append(comm_norm)
    commutator_results.append(comm_vec)
    
    print(f"  ||[Htf, Hi] |ψ_{i}⟩|| = {comm_norm:.2e}")
    
    # Check if it's effectively zero
    if comm_norm < 1e-10:
        print(f"  ✓ Commutator effectively zero on this eigenvector")
        
        # Verify this is a simultaneous eigenvector
        Hi_vec = Hi @ vec
        Htf_vec = Htf @ vec
        
        # Check if Hi_vec is proportional to vec
        nonzero_mask = np.abs(vec) > 1e-12
        if np.any(nonzero_mask):
            Hi_ratios = Hi_vec[nonzero_mask] / vec[nonzero_mask]
            Hi_ratio_std = np.std(Hi_ratios)
            Hi_eigenval = np.mean(Hi_ratios)
            
            if Hi_ratio_std < 1e-8:
                print(f"    Hi eigenvalue: {Hi_eigenval:.6f}")
                
                # Check if Htf_vec is proportional to vec
                Htf_ratios = Htf_vec[nonzero_mask] / vec[nonzero_mask]
                Htf_ratio_std = np.std(Htf_ratios)
                Htf_eigenval = np.mean(Htf_ratios)
                
                if Htf_ratio_std < 1e-8:
                    print(f"    Htf eigenvalue: {Htf_eigenval:.6f}")
                    print(f"    Total H eigenvalue: {Hi_eigenval + Htf_eigenval:.6f}")
                    print(f"    Expected H eigenvalue: {eigenval:.6f}")
                    print(f"    Match: {abs(Hi_eigenval + Htf_eigenval - eigenval) < 1e-6}")
                else:
                    print(f"    Not a clean Htf eigenvector (std: {Htf_ratio_std:.2e})")
            else:
                print(f"    Not a clean Hi eigenvector (std: {Hi_ratio_std:.2e})")
    else:
        print(f"  ❌ Commutator NOT zero on this eigenvector")
        
        # Show the largest components of the commutator result
        largest_indices = np.argsort(np.abs(comm_vec))[-5:]
        print(f"  Largest commutator components:")
        for idx in largest_indices:
            binary_state = format(idx, f'0{N}b')
            print(f"    |{binary_state}⟩: {comm_vec[idx]:.6f}")

print(f"\nSummary:")
print(f"Commutator norms: {commutator_norms}")
zero_comm_count = sum(1 for norm in commutator_norms if norm < 1e-10)
print(f"Number of eigenvectors with effectively zero commutator: {zero_comm_count}/{len(sparse_indices)}")

# Optional: Check if the commutator itself has any special structure
print(f"\nCommutator properties:")
print(f"  Is Hermitian: {np.allclose(commutator.toarray(), commutator.toarray().conj().T)}")
print(f"  Frobenius norm: {np.linalg.norm(commutator.toarray(), ord='fro'):.2e}")

In [ ]:
# Check if sparse eigenstates are simultaneous eigenvectors of Htf and Hi
print("Analyzing simultaneous eigenstate properties of sparse eigenvectors...")

# Build the Hamiltonians
Hi = ising_icosahedron(N, J)
Htf = transverse_field_icosahedral(N, h)

# Analyze each sparse eigenvector
simultaneous_eigenstates = []
eigenvalue_analysis = []

for idx, i in enumerate(sparse_indices):
    vec = eigenvectors[:, i]
    H_eigenval = eigenvalues[i]
    
    print(f"\n{'='*60}")
    print(f"Analyzing eigenvector {i} (H eigenvalue: {H_eigenval:.8f})")
    print(f"{'='*60}")
    
    # Test if it's an eigenvector of Hi (Ising term)
    Hi_vec = Hi @ vec
    nonzero_mask = np.abs(vec) > 1e-12
    
    if np.any(nonzero_mask):
        Hi_ratios = Hi_vec[nonzero_mask] / vec[nonzero_mask]
        Hi_ratio_std = np.std(Hi_ratios)
        Hi_eigenval = np.mean(Hi_ratios)
        
        print(f"Hi analysis:")
        print(f"  Mean eigenvalue: {Hi_eigenval:.8f}")
        print(f"  Ratio std: {Hi_ratio_std:.2e}")
        print(f"  Is integer? {abs(Hi_eigenval - round(Hi_eigenval)) < 1e-8}")
        if abs(Hi_eigenval - round(Hi_eigenval)) < 1e-8:
            print(f"  Rounded eigenvalue: {round(Hi_eigenval)}")
        
        Hi_is_eigenstate = Hi_ratio_std < 1e-10
        print(f"  Is Hi eigenstate: {Hi_is_eigenstate}")
        
        # Test if it's an eigenvector of Htf (transverse field term)
        Htf_vec = Htf @ vec
        if np.any(nonzero_mask):
            Htf_ratios = Htf_vec[nonzero_mask] / vec[nonzero_mask]
            Htf_ratio_std = np.std(Htf_ratios)
            Htf_eigenval = np.mean(Htf_ratios)
            
            print(f"\nHtf analysis:")
            print(f"  Mean eigenvalue: {Htf_eigenval:.8f}")
            print(f"  Ratio std: {Htf_ratio_std:.2e}")
            print(f"  Is Htf eigenstate: {Htf_ratio_std < 1e-10}")
            
            Htf_is_eigenstate = Htf_ratio_std < 1e-10
            
            # Check if it's a simultaneous eigenstate
            if Hi_is_eigenstate and Htf_is_eigenstate:
                expected_H_eigenval = Hi_eigenval + Htf_eigenval
                eigenval_match = abs(expected_H_eigenval - H_eigenval) < 1e-8
                
                print(f"\n*** SIMULTANEOUS EIGENSTATE FOUND ***")
                print(f"  Hi eigenvalue: {Hi_eigenval:.8f}")
                print(f"  Htf eigenvalue: {Htf_eigenval:.8f}")
                print(f"  Expected H eigenvalue: {expected_H_eigenval:.8f}")
                print(f"  Actual H eigenvalue: {H_eigenval:.8f}")
                print(f"  Eigenvalue match: {eigenval_match}")
                
                if eigenval_match:
                    simultaneous_eigenstates.append(i)
                    eigenvalue_analysis.append({
                        'index': i,
                        'H_eigenval': H_eigenval,
                        'Hi_eigenval': Hi_eigenval,
                        'Htf_eigenval': Htf_eigenval,
                        'Hi_ratio_std': Hi_ratio_std,
                        'Htf_ratio_std': Htf_ratio_std
                    })
                    
                    # Additional analysis for simultaneous eigenstates
                    print(f"\n  Additional properties:")
                    print(f"    Hi eigenvalue is integer: {abs(Hi_eigenval - round(Hi_eigenval)) < 1e-8}")
                    print(f"    Number of nonzero components: {np.sum(nonzero_mask)}")
                    
                    # Check commutator [Hi, Htf] acting on this vector
                    commutator_vec = (Hi @ Htf @ vec) - (Htf @ Hi @ vec)
                    comm_norm = np.linalg.norm(commutator_vec)
                    print(f"    ||[Hi, Htf] |ψ⟩||: {comm_norm:.2e}")
                    
            elif Hi_is_eigenstate and not Htf_is_eigenstate:
                print(f"\n  → Only Hi eigenstate (eigenvalue: {Hi_eigenval:.8f})")
                print(f"  → NOT Htf eigenstate (std: {Htf_ratio_std:.2e})")
                
                # Show some sample Htf ratios for debugging
                print(f"  Sample Htf ratios (first 5 significant components):")
                sample_indices = np.where(nonzero_mask)[0][:5]
                for j, idx in enumerate(sample_indices):
                    binary_state = format(idx, f'0{N}b')
                    if j < len(Htf_ratios):
                        print(f"    |{binary_state}⟩: {Htf_ratios[j]:.8f}")
                        
            elif not Hi_is_eigenstate and Htf_is_eigenstate:
                print(f"\n  → Only Htf eigenstate (eigenvalue: {Htf_eigenval:.8f})")
                print(f"  → NOT Hi eigenstate (std: {Hi_ratio_std:.2e})")
                
                # Show some sample Hi ratios for debugging
                print(f"  Sample Hi ratios (first 5 significant components):")
                sample_indices = np.where(nonzero_mask)[0][:5]
                for j, idx in enumerate(sample_indices):
                    binary_state = format(idx, f'0{N}b')
                    if j < len(Hi_ratios):
                        print(f"    |{binary_state}⟩: {Hi_ratios[j]:.8f}")
                        
            else:
                print(f"\n  → NOT an eigenstate of either Hi or Htf")
                print(f"  → Hi ratio std: {Hi_ratio_std:.2e}")
                print(f"  → Htf ratio std: {Htf_ratio_std:.2e}")
                
        else:
            print(f"  No significant nonzero components in Htf calculation")
    else:
        print(f"  No significant nonzero components in vector")

# Summary
print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Total sparse indices analyzed: {len(sparse_indices)}")
print(f"Simultaneous eigenstates found: {len(simultaneous_eigenstates)}")

if simultaneous_eigenstates:
    print(f"\nSimultaneous eigenstate indices: {simultaneous_eigenstates}")
    print(f"\nDetailed eigenvalue breakdown:")
    for analysis in eigenvalue_analysis:
        print(f"  Index {analysis['index']}:")
        print(f"    H  = {analysis['H_eigenval']:.8f}")
        print(f"    Hi = {analysis['Hi_eigenval']:.8f} (std: {analysis['Hi_ratio_std']:.2e})")
        print(f"    Htf= {analysis['Htf_eigenval']:.8f} (std: {analysis['Htf_ratio_std']:.2e})")
        print(f"    Hi + Htf = {analysis['Hi_eigenval'] + analysis['Htf_eigenval']:.8f}")
        print(f"    Difference: {abs(analysis['H_eigenval'] - (analysis['Hi_eigenval'] + analysis['Htf_eigenval'])):.2e}")
        
    # Check for patterns in the eigenvalues
    Hi_eigenvals = [a['Hi_eigenval'] for a in eigenvalue_analysis]
    Htf_eigenvals = [a['Htf_eigenval'] for a in eigenvalue_analysis]
    
    print(f"\nEigenvalue patterns:")
    print(f"  Hi eigenvalues: {Hi_eigenvals}")
    print(f"  Htf eigenvalues: {Htf_eigenvals}")
    print(f"  Unique Hi eigenvalues: {list(set(Hi_eigenvals))}")
    print(f"  Unique Htf eigenvalues: {list(set(Htf_eigenvals))}")
    
else:
    print(f"No simultaneous eigenstates found among the sparse indices.")
    print(f"This suggests the quantum scars do not simultaneously diagonalize both terms.")

In [ ]:
# scars + tf field - sparse


for i in tqdm(sparse_indices):
    # Construct the density matrix for each eigenvector (as sparse)
    min_eigenvector = eigenvectors[:, i]
    print("Nonzero elements in state vector (tol=1e-12):", np.sum(np.abs(min_eigenvector) > 1e-6))
    min_eigenvector[np.abs(min_eigenvector) < 1e-7] = 0
    min_eig_sparse = csr_matrix(min_eigenvector.reshape(-1, 1))  # Convert to sparse column vector
    
    # Apply the ising operator to the sparse vector
    vecs = ising_icosahedron(N, J) @ min_eig_sparse
    vecs = vecs.toarray()  # Convert to dense array for further processing
    
    # Print information about the result
    print(f"Shape of result: {vecs.shape}")
    print(f"for {i}-th eigenvector: Number of nonzero elements in result: {np.count_nonzero(vecs)}")
    print(f"Max absolute value in result: {np.max(np.abs(vecs.data))}")

In [ ]:
# Analyze eigenvector 1527: expectation values and action of transverse field term

idx = 2057
vec = eigenvectors[:, idx]

# Build Ising icosahedron Hamiltonian (no transverse field)
H_ising = ising_icosahedron(N, J)

# Build transverse field Hamiltonian only
H_tf = transverse_field_icosahedral(N, h)

# Compute expectation values
eig_ising = np.vdot(vec, H_ising @ vec)
eig_tf = np.vdot(vec, H_tf @ vec)

print(f"Eigenvector {idx}:")
print(f"  Expectation value w.r.t. Ising icosahedron term: {eig_ising}")
print(f"  Expectation value w.r.t. transverse field term: {eig_tf}")

# Analyze H_tf @ vec
tf_vec = H_tf @ vec
print(f"\nAnalysis of H_tf @ vec for eigenvector {idx}:")
print(f"  Norm: {np.linalg.norm(tf_vec)}")
print(f"  Max abs value: {np.max(np.abs(tf_vec))}")
print(f"  Number of nonzero components (>1e-12): {np.sum(np.abs(tf_vec) > 1e-12)}")
print(f"  Is tf_vec proportional to vec? Ratio: {np.allclose(tf_vec, eig_tf * vec)}")

# Analyze H_ising @ vec
ising_vec = H_ising @ vec
print(f"\nAnalysis of H_ising @ vec for eigenvector {idx}:")
print(f"  Norm: {np.linalg.norm(ising_vec)}")
print(f"  Max abs value: {np.max(np.abs(ising_vec))}")
print(f"  Number of nonzero components (>1e-12): {np.sum(np.abs(ising_vec) > 1e-12)}")
print(f"  Is ising_vec proportional to vec? Ratio: {np.allclose(ising_vec, eig_ising * vec)}")

# Check orthogonality of vec and H_tf @ vec

inner_product = np.vdot(vec, tf_vec)
print(f"Inner product <vec|H_tf|vec>: {inner_product}")
print(f"Is orthogonal (abs(inner_product) < 1e-12)? {np.abs(inner_product) < 1e-12}")

In [ ]:
# Assuming ising_vec and vec are numpy arrays
def check_proportional(ising_vec, vec, tol=1e-8):
    # Find indices where vec is not zero to avoid division by zero
    nonzero_indices = np.where(np.abs(vec) > tol)[0]
    if len(nonzero_indices) == 0:
        return False, None  # Cannot determine proportionality

    # Calculate proportionality constants for nonzero elements
    ratios = ising_vec[nonzero_indices] / vec[nonzero_indices]
    # Check if all ratios are (almost) equal
    if np.allclose(ratios, ratios[0], atol=tol, rtol=tol):
        constant = ratios[0]
        # Optionally, check the full vector proportionality
        if np.allclose(ising_vec, constant * vec, atol=tol, rtol=tol):
            return True, constant
    return False, None

# Example usage:
is_proportional, constant = check_proportional(ising_vec, vec)
if is_proportional:
    print(f"ising_vec is proportional to vec with constant {constant}")
else:
    print("ising_vec is not proportional to vec")

In [ ]:
##################### RDMS + EE #################################

In [ ]:
'''# rdm - qutip

min_eigenvalues = []
min_rdms = []
rdm_eigenvalues = []

for i in tqdm(range(len(eigenvalues))):
    # Construct the density matrix for each eigenvector
    min_eigenvector = eigenvectors[:, i]
    density_matrix = np.outer(min_eigenvector, min_eigenvector.conj())
    density_matrix_qobj = Qobj(density_matrix, dims=[[2]*N, [2]*N])

    # Trace out qubits
    traced_out_density_matrix = ptrace(density_matrix_qobj, keep_qubits)

    # Convert the result back to a dense matrix if needed
    traced_out_density_matrix_dense = traced_out_density_matrix.full()
    # Check if the rank is not full and print it
    rank = np.linalg.matrix_rank(traced_out_density_matrix_dense)
    if rank < traced_out_density_matrix_dense.shape[0]:
        print(f"Traced-out density matrix rank: {rank}")
    # Diagonalize the traced-out density matrix
    eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(traced_out_density_matrix_dense)

    # Find the minimum eigenvalue of the traced-out density matrix
    min_eigenvalue = np.min(eigenvalues_traced) 
    min_eigenvalues.append(min_eigenvalue)

    if min_eigenvalue < 1e-16:
      min_rdms.append(traced_out_density_matrix_dense) #store the scarred rdms - step needed for optimization
      rdm_eigenvalues.append(eigenvalues_traced) #store the eigenvalues of the scarred rdms# Define the threshold - qutip
threshold = 1e-16
min_eigenvalues = np.array(min_eigenvalues)

# Count points with y-component less than the threshold
count = sum(1 for y in min_eigenvalues if y < threshold)

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues]

# Plot the minimum eigenvalue of the traced-out density matrix as a function of the eigenvalue of the Hamiltonian H
plt.scatter(eigenvalues, np.abs(min_eigenvalues), color=colors, s=1)

plt.xlabel('Eigenvalue of H')
plt.ylabel(r"$\lambda_{\text{min}}$")
plt.yscale("log")
plt.show()

# Find indices of red points - qutip
red_indices = [i for i, y in enumerate(min_eigenvalues) if y < threshold]
print(f'Indices of red points: {red_indices}')

red_eigenvectors = []

# Print eigenvalues, RDM minimum eigenvalues, and scalar products of eigenvectors for red points
for i,ind in enumerate(red_indices):
    print(i, ind)
    print(f'Eigenvalue: {eigenvalues[ind]}, rdm Minimum Eigenvalue: {min_eigenvalues[ind]}')
    print(f'Eigenvector {ind}: rdm rank: {np.linalg.matrix_rank(min_rdms[i])}')
    red_eigenvectors.append(eigenvectors[:, ind])

for i in range(len(red_indices)):
    for j in range(len(red_indices)):
        idx1, idx2 = red_indices[i], red_indices[j]
        dot_product = np.dot(eigenvectors[:, idx1], np.conj(eigenvectors[:, idx2]))
        print(f"Dot product between eigenvectors {idx1} and {idx2}: {dot_product}")

# Count entries of rdm_eigenvalues[i] - eigenvalues of the scarred rdms - that are non-zero
counts = [np.sum(eigenvalues > 1e-16) for eigenvalues in rdm_eigenvalues]
print(f'Counts of non-zero eigenvalues of the scarred rdms: {counts}')'''

In [ ]:
# rdm - sparse

min_eigenvalues_sparse = []
min_rdms_sparse = []
rdm_ranks_sparse = []

red_indices_sparse = sparse_indices
#red_indices_sparse = [1266,1267,1268,1269,1270, 1527] # 5 scars for 5 spins rdm with keep_qubits = [2,4,5,8,9] and h=3J

for i in tqdm(red_indices_sparse):
    # Construct the density matrix for each eigenvector (as sparse)
    min_eigenvector = eigenvectors[:, i]
    print("Nonzero elements in state vector (tol=1e-12):", np.sum(np.abs(min_eigenvector) > 1e-6))
    min_eigenvector[np.abs(min_eigenvector) < 1e-7] = 0
    min_eig_sparse = csr_matrix(min_eigenvector.reshape(-1, 1))  # Convert to sparse column vector
    density_matrix_sparse = min_eig_sparse @ min_eig_sparse.getH()  # Outer product to form density matrix

    print("Number of nonzero elements of dm (tol=1e-12):", np.sum(np.abs(density_matrix_sparse.data) > 1e-6))    #Trace out qubits using qutip partial trace
    rdm = ptrace_sparse(density_matrix_sparse, keep_qubits, [2]*N) # Use the custom ptrace_sparse function
    # Find the minimum eigenvalue of the traced-out density matrix
    eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.toarray())
    min_eigenvalue = np.min(eigenvalues_traced)
    min_eigenvalues_sparse.append(min_eigenvalue)


    min_rdms_sparse.append(rdm)  # store the sparse RDM
    rdm_ranks_sparse.append(np.linalg.matrix_rank(rdm.toarray()))  # store the rank

# Define the threshold
threshold = 1e-16
min_eigenvalues_sparse = np.array(min_eigenvalues_sparse)

# Count points with y-component less than the threshold
count = sum(1 for y in min_eigenvalues_sparse if y < threshold)

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_sparse]

x = eigenvalues[red_indices_sparse]
print(f"x: {x}")
# Plot the minimum eigenvalue of the traced-out density matrix as a function of the eigenvalue of the Hamiltonian H
plt.scatter(x, np.abs(min_eigenvalues_sparse), color=colors, s=1)
#plt.xlim(-7,-5)

plt.xlabel('Eigenvalue of H')
plt.ylabel(r"$\lambda_{\text{min}}$")
plt.yscale("log")
plt.show()

# Find indices of red points - sparse
#red_indices_sparse = [i for i, y in enumerate(min_eigenvalues_sparse) if y < threshold]
#print(f'Indices of red points: {red_indices_sparse}')

red_eigenvectors_sparse = []

# Print eigenvalues, RDM minimum eigenvalues, and ranks for red points
for i, ind in enumerate(red_indices_sparse):
    print(i, ind)
    print(f'Eigenvalue: {eigenvalues[ind]}, rdm Minimum Eigenvalue: {min_eigenvalues_sparse[i]}')
    print(f'Eigenvector {ind}: rdm rank: {rdm_ranks_sparse[i]}')
    red_eigenvectors_sparse.append(eigenvectors[:, ind])

# Print the ranks of the scarred rdms
print(f'Ranks of the scarred rdms: {rdm_ranks_sparse}')

In [ ]:
# scars + tf field - sparse

red_indices_np = sparse_indices #[1266,1267,1268,1269,1270]

for i in tqdm(red_indices_np):
    # Construct the density matrix for each eigenvector (as sparse)
    min_eigenvector = eigenvectors[:, i]
    print("Nonzero elements in state vector (tol=1e-12):", np.sum(np.abs(min_eigenvector) > 1e-6))
    min_eigenvector[np.abs(min_eigenvector) < 1e-7] = 0
    min_eig_sparse = csr_matrix(min_eigenvector.reshape(-1, 1))  # Convert to sparse column vector
    
    # Apply the transverse field operator to the sparse vector
    vecs = transverse_field_icosahedral(N, h) @ min_eig_sparse
    vecs = vecs.toarray()  # Convert to dense array for further processing
    
    # Print information about the result
    print(f"Shape of result: {vecs.shape}")
    print(f"for {i}-th eigenvector: Number of nonzero elements in result: {np.count_nonzero(vecs)}")
    print(f"Max absolute value in result: {np.max(np.abs(vecs.data))}")

In [ ]:
# Compute commutator [H_tf, ρ_scar] for each scarred state

H_tf = transverse_field_icosahedral(N, h)  # Transverse field Hamiltonian (sparse)
commutator_norms = []

for i in red_indices_sparse:
    scar_vec = eigenvectors[:, i]
    # Density matrix of the scarred state
    rho_scar = np.outer(scar_vec, scar_vec.conj())
    # Convert to sparse for multiplication
    rho_scar_sparse = csr_matrix(rho_scar)
    # Compute commutator: [H_tf, ρ_scar] = H_tf * ρ_scar - ρ_scar * H_tf
    comm = H_tf @ rho_scar_sparse - rho_scar_sparse @ H_tf
    # Compute the Frobenius norm of the commutator
    comm_norm = np.linalg.norm(comm.toarray(), ord='fro')
    commutator_norms.append(comm_norm)
    print(f"Scar index {i}: ||[H_tf, ρ_scar]||_F = {comm_norm:.3e}")

print("All commutator norms:", commutator_norms)

In [ ]:
# Compute commutator [H_tf_sub, RDM] for each scarred state, all in sparse, using ptrace_sparse

def transverse_field_subsystem_sparse(n, h):
    """Transverse field Hamiltonian for n spins (subsystem), sparse."""
    Z = csr_matrix(np.array([[1, 0], [0, -1]]))
    I = identity(2, format="csr")
    H = csr_matrix((2**n, 2**n), dtype=np.complex128)
    for i in range(n):
        ops = [I] * n
        ops[i] = Z
        term = ops[0]
        for op in ops[1:]:
            term = kron(term, op, format="csr")
        H += -h * term
    return H

some_indices_nnsparse =  [1275]
n_sub = len(keep_qubits)
H_tf_sub_sparse = transverse_field_subsystem_sparse(n_sub, h)

# Compute RDMs for all scarred states using ptrace_sparse (all sparse)
rdms_sparse = []
for i in red_indices_sparse:
    scar_vec = eigenvectors[:, i]
    scar_vec[np.abs(scar_vec) < 1e-7] = 0
    scar_vec_sparse = csr_matrix(scar_vec.reshape(-1, 1))
    rho_sparse = scar_vec_sparse @ scar_vec_sparse.getH()
    rdm_sparse = ptrace_sparse(rho_sparse, keep_qubits, [2]*N)
    rdms_sparse.append(rdm_sparse)

# Compute commutator norms
commutator_norms_rdm_sparse = []
for idx, rdm in enumerate(rdms_sparse):
    comm = H_tf_sub_sparse @ rdm - rdm @ H_tf_sub_sparse
    comm_norm = np.linalg.norm(comm.toarray(), ord='fro')
    print(f"Scar RDM {idx} (index {red_indices_sparse[idx]}): ||[H_tf_sub, RDM]||_F = {comm_norm:.3e}")
    commutator_norms_rdm_sparse.append(comm_norm)

print("All commutator norms for sparse RDMs:", commutator_norms_rdm_sparse)

In [ ]:
# rdm - numpy

min_eigenvalues_np = []
min_rdms_np =[]
rdm_eigenvalues_np = []

for i in tqdm(range(len(eigenvalues))):
    # Construct the density matrix for each eigenvector
    min_eigenvector = eigenvectors[:, i]
    density_matrix = np.outer(min_eigenvector, min_eigenvector.conj())

    # Trace out qubits
    traced_out_density_matrix = ptrace_numpy(density_matrix, keep_qubits, [[2]*N, [2]*N])

    # Diagonalize the traced-out density matrix
    eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(traced_out_density_matrix)

    # Find the minimum eigenvalue of the traced-out density matrix
    min_eigenvalue = np.min(eigenvalues_traced) 
    min_eigenvalues_np.append(min_eigenvalue)

    if min_eigenvalue < 1e-16:
        min_rdms_np.append(traced_out_density_matrix) #store the scarred rdms - step needed for optimization
        rdm_eigenvalues_np.append(eigenvalues_traced) #store the eigenvalues of the scarred rdms

# numpy

# Define the threshold
threshold = 1e-16
min_eigenvalues_np = np.array(min_eigenvalues_np)

# Count points with y-component less than the threshold
count = sum(1 for y in min_eigenvalues_np if y < threshold)

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_np]

# Plot the minimum eigenvalue of the traced-out density matrix as a function of the eigenvalue of the Hamiltonian H
plt.scatter(eigenvalues, np.abs(min_eigenvalues_np), color=colors, s=1)

plt.xlabel('Eigenvalue of H')
plt.ylabel(r"$\lambda_{\text{min}}$")
plt.yscale("log")
plt.show()

# Find indices of red points - numpy
red_indices_np = [i for i, y in enumerate(min_eigenvalues_np) if y < threshold]
print(f'Indices of red points: {red_indices_np}')

red_eigenvectors_np = []

# Print eigenvalues, RDM minimum eigenvalues, and scalar products of eigenvectors for red points
for i,ind in enumerate(red_indices_np):
    print(i, ind)
    print(f'Eigenvalue: {eigenvalues[ind]}, rdm Minimum Eigenvalue: {min_eigenvalues_np[ind]}')
    print(f'Eigenvector {ind}: rdm rank: {np.linalg.matrix_rank(min_rdms_np[i])}')
    red_eigenvectors_np.append(eigenvectors[:, ind])

#for i in range(len(red_indices_np)):
#    for j in range(len(red_indices_np)):
#        idx1, idx2 = red_indices_np[i], red_indices_np[j]
#        dot_product = np.dot(eigenvectors[:, idx1], np.conj(eigenvectors[:, idx2]))
#        print(f"Dot product between eigenvectors {idx1} and {idx2}: {dot_product}")

# Count entries of rdm_eigenvalues_np[i] - eigenvalues of the scarred rdms - that are non-zero
counts_np = [np.sum(eigenvalues > 1e-16) for eigenvalues in rdm_eigenvalues_np]
print(f'Counts of non-zero eigenvalues of the scarred rdms: {counts_np}')

In [ ]:
'''# rdm - torch

min_eigenvalues_to = []
min_rdms_to = []
rdm_eigenvalues_to = []

for i in tqdm(range(len(eigenvalues))):
    # Construct the density matrix for each eigenvector
    min_eigenvector = eigenvectors[:, i]
    min_eigenvector_torch = torch.tensor(min_eigenvector, dtype=torch.float64)
    density_matrix = torch.outer(min_eigenvector_torch, min_eigenvector_torch.conj())

    # Trace out qubits
    traced_out_density_matrix = ptrace_torch(density_matrix, keep_qubits, [[2] * N, [2] * N])
    traced_out_density_matrix = torch.tensor(traced_out_density_matrix, dtype=torch.float64) # necessary step to ensure that the scarred states have lambda_min of the correct order - <1e-16 

    # Diagonalize the traced-out density matrix
    eigenvalues_traced, eigenvectors_traced = torch.linalg.eigh(traced_out_density_matrix)

    # Find the minimum eigenvalue of the traced-out density matrix
    min_eigenvalue = torch.min(eigenvalues_traced).item()
    min_eigenvalues_to.append(min_eigenvalue)

    if min_eigenvalue < 1e-16:
        min_rdms_to.append(traced_out_density_matrix.detach().cpu().numpy())  # store the scarred rdms - step needed for optimization
        rdm_eigenvalues_to.append(eigenvalues_traced.detach().cpu().numpy()) # store the eigenvalues of the scarred rdms

# Define the threshold
threshold = 1e-16
min_eigenvalues_to = np.array(min_eigenvalues_to)

# Count points with y-component less than the threshold
count = sum(1 for y in min_eigenvalues_to if y < threshold)

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_to]

# Plot the minimum eigenvalue of the traced-out density matrix as a function of the eigenvalue of the Hamiltonian H
plt.scatter(eigenvalues, np.abs(min_eigenvalues_to), color=colors, s=1)

plt.xlabel('Eigenvalue of H')
plt.ylabel(r"$\lambda_{\text{min}}$")
plt.yscale("log")
plt.show()

# Find indices of red points - torch
red_indices_to = [i for i, y in enumerate(min_eigenvalues_to) if y < threshold]
print(f'Indices of red points: {red_indices_to}')

red_eigenvectors_to = []

# Print eigenvalues, RDM minimum eigenvalues, and scalar products of eigenvectors for red points
for i,ind in enumerate(red_indices_to):
    print(i, ind)
    print(f'Eigenvalue: {eigenvalues[ind]}, rdm Minimum Eigenvalue: {min_eigenvalues_to[ind]}')
    print(f'Eigenvector {ind}: rdm rank: {np.linalg.matrix_rank(min_rdms_to[i])}')
    red_eigenvectors_to.append(eigenvectors[:, ind])

for i in range(len(red_indices_to)):
    for j in range(len(red_indices_to)):
        idx1, idx2 = red_indices_to[i], red_indices_to[j]
        dot_product = np.dot(eigenvectors[:, idx1], np.conj(eigenvectors[:, idx2]))
        print(f"Dot product between eigenvectors {idx1} and {idx2}: {dot_product}")

# Count entries of rdm_eigenvalues_to[i] - eigenvalues of the scarred rdms - that are non-zero
counts_to = [np.sum(eigenvalues > 1e-15) for eigenvalues in rdm_eigenvalues_to]
print(f'Counts of non-zero eigenvalues of the scarred rdms: {counts_to}')'''

In [ ]:
half_v = [0,2,4,5,8,9]
half_h = [0,2,5,7,9,11]

half = half_h

In [ ]:
# Generate random complex coefficients
random_coeffs = np.random.rand(len(red_indices_np)) + 1j* np.random.rand(len(red_indices_np))

# Normalize the coefficients
random_coeffs /= np.linalg.norm(random_coeffs)
print(random_coeffs)

print("coeffs norm",sum(np.abs(random_coeffs)**2))

# Define random scar state from 5 scarred states using random coefficients
rnd_scar = np.sum(eigenvectors[:, red_indices_np] * random_coeffs, axis=1)

print(len(rnd_scar))

# Save eigenvalues corresponding to red indices into a numpy array
red_eigenvalues = np.array([eigenvalues[i] for i in red_indices_np])

# Count the number of zero entries
num_zeros = np.count_nonzero((rnd_scar.real < 1e-16) & (rnd_scar.imag < 1e-16))
print(f'Number of zero entries: {num_zeros}')

# RANDOM SCAR - RANDOM LINEAR COMBINATION OF SCARRED STATES

rnd_scar = linear_combination_np(random_coeffs, red_eigenvectors_np)
rnd_scar_entropy = entanglement_entropy(rnd_scar, half, N)
#rnd_scar_entropy_qutip = entanglement_entropy_qutip(rnd_scar, half, N)
#rnd_scar_entropy_torch = entanglement_entropy_torch(rnd_scar, half, N)
print("Random scar coeffs", random_coeffs)
print(f"Entanglement entropy of random scar state: {rnd_scar_entropy}")
#print(f"Entanglement entropy of random scar state (Qutip): {rnd_scar_entropy_qutip}")
#print(f"Entanglement entropy of random scar state (PyTorch): {rnd_scar_entropy_torch}")

In [ ]:
# Compute bipartite entanglement entropy for all eigenstates in the full Hilbert space
numpy_entropies = [entanglement_entropy(eigenvectors[:, i], half, N) for i in tqdm(range(eigenvectors.shape[1]))]
#qutip_entropies = [entanglement_entropy_qutip(eigenvectors[:, i], half, N) for i in tqdm(range(eigenvectors.shape[1]))]
#torch_entropies = [entanglement_entropy_torch(eigenvectors[:, i], half, N) for i in tqdm(range(eigenvectors.shape[1]))]

In [ ]:
# Highlight selected eigenvalues in red on a plot

plt.scatter(eigenvalues, numpy_entropies, color='C0', s=1, label='numpy')
plt.scatter([eigenvalues[i] for i in sparse_indices],
            [numpy_entropies[i] for i in sparse_indices],
            color='red', s=20, label='Scars')
plt.legend()
plt.show()

In [ ]:
# Define a colormap for red points
red_colors = plt.cm.rainbow(np.linspace(0, 1, len(red_indices_np)))

fig, ax = plt.subplots(figsize=(11, 6))

# Plot all points (blue for non-red)
colors = ['red' if i in red_indices_np else 'C0' for i in range(len(eigenvalues))]
for i, color in enumerate(colors):
    ax.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color if color == 'C0' else 'none', markersize=2)

# Plot each red point with a different color and collect handles/labels for legend
handles = []
labels = []
for i, idx in enumerate(red_indices_np):
    handle, = ax.plot(eigenvalues[idx], numpy_entropies[idx], 'o', color=red_colors[i], markersize=7)
    handles.append(handle)
    labels.append(f'Eigenvalue: {eigenvalues[idx]:.2f}, Entropy: {numpy_entropies[idx]:.2f}')

# Add legend for each red point
ax.legend(handles, labels, title="Red Points (Scars)", loc='best', fontsize=9)

ax.set_xlabel('Eigenvalue of H')
ax.set_ylabel('Entanglement entropy')
plt.show()

In [ ]:
# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices_np:
    print(f'Index: {i}, Entropy: {numpy_entropies[i]}')

print(f'Random scar entropy: {rnd_scar_entropy}')

# Plot entanglement entropy as a function of energy eigenvalues
plt.figure(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y in sparse_indices else 'C0' for y in min_eigenvalues_np]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    plt.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=2)

# Plot the maximum scar entropy in yellow
plt.plot(np.mean(red_eigenvalues), rnd_scar_entropy, 'x', color='yellow', markersize=7, label='Random Scar')

# Add legend for red points only with text "scars"
red_points = [i for i, color in enumerate(colors) if color == 'red']
if red_points:
    plt.plot([], [], 'o', color='red', label='Scars')

plt.xlabel('Eigenvalue of H')
plt.ylabel('Entanglement entropy')
plt.legend()
plt.show()

In [ ]:
# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices_np:
    print(f'Index: {i}, Entropy: {numpy_entropies[i]}')

# Find the red index with the lowest entanglement entropy
min_entropy_idx = min(red_indices_np, key=lambda i: numpy_entropies[i])
min_entropy_val = numpy_entropies[min_entropy_idx]

# Plot entanglement entropy as a function of energy eigenvalues
plt.figure(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y in sparse_indices else 'C0' for y in min_eigenvalues_np]

# Plot blue points first
for i, color in enumerate(colors):
    if color != 'red':
        plt.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=2)
# Then plot red points on top
for i, color in enumerate(colors):
    if color == 'red':
        plt.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=4)

# Plot entanglement entropy with highlighted points
#for i, color in enumerate(colors):
#    size = 3 if color == 'red' else 2
#    plt.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=size)

# Mark the scar with the lowest entanglement entropy with a yellow cross
plt.plot(eigenvalues[min_entropy_idx], min_entropy_val, 'x', color='red', markersize=8, label=f'Min EE (idx={min_entropy_idx + 1})')

# Add legend for red points only with text "scars"
red_points = [i for i, color in enumerate(colors) if color == 'red']
if red_points:
    plt.plot([], [], 'o', color='red', markersize=4, label='Scars')

plt.xlabel('Eigenvalue of H')
plt.ylabel('Entanglement entropy')
plt.legend()
plt.show()

In [ ]:
'''# Compute entanglement entropy for the rnd_scar wavefunction - qutip

# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices:
    print(f'Index: {i}, Entropy: {qutip_entropies[i]}')

print(f'Random scar entropy: {rnd_scar_entropy_qutip}')

# Plot entanglement entropy as a function of energy eigenvalues
plt.figure(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    plt.plot(eigenvalues[i], qutip_entropies[i], 'o', color=color, markersize=2)

# Plot the maximum scar entropy in yellow
plt.plot(np.mean(red_eigenvalues), rnd_scar_entropy_qutip, 'x', color='yellow', markersize=7, label='Random Scar')

# Add legend for red points only with text "scars"
red_points = [i for i, color in enumerate(colors) if color == 'red']
if red_points:
    plt.plot([], [], 'o', color='red', label='Scars')

plt.xlabel('Eigenvalue of H')
plt.ylabel('Entanglement entropy')
#plt.title('Entanglement entropy as a function of energy eigenvalues')
plt.legend()
plt.show()'''

In [ ]:
'''# Compute entanglement entropy for the rnd_scar wavefunction - torch

# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices:
    print(f'Index: {i}, Entropy: {torch_entropies[i]}')

print(f'Random scar entropy: {rnd_scar_entropy_torch}')

# Plot entanglement entropy as a function of energy eigenvalues
plt.figure(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    plt.plot(eigenvalues[i], torch_entropies[i], 'o', color=color, markersize=2)

# Plot the maximum scar entropy in yellow
plt.plot(np.mean(red_eigenvalues), rnd_scar_entropy_torch, 'x', color='yellow', markersize=7, label='Random Scar')

# Add legend for red points only with text "scars"
red_points = [i for i, color in enumerate(colors) if color == 'red']
if red_points:
    plt.plot([], [], 'o', color='red', label='Scars')

plt.xlabel('Eigenvalue of H')
plt.ylabel('Entanglement entropy')
#plt.title('Entanglement entropy as a function of energy eigenvalues')
plt.legend()
plt.show()'''

In [ ]:
# Define a list of colors for red points
red_colors = plt.cm.rainbow(np.linspace(0, 1, len(red_indices_np)))

# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices_np:
    print(f'Index: {i}, Entropy: {numpy_entropies[i]}')

print(f'Random scar entropy: {rnd_scar_entropy}')

# Plot entanglement entropy as a function of energy eigenvalues
fig, ax = plt.subplots(figsize=(11, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_np]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    ax.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=2)

# Collect handles and labels for the legend
handles = []
labels = []

# Plot each red point with a different color and collect handles and labels
for i, idx in enumerate(red_indices_np):
    handle, = ax.plot(eigenvalues[idx], numpy_entropies[idx], 'o', color=red_colors[i], markersize=5)
    handles.append(handle)
    labels.append(f'λ_min: {float(min_eigenvalues_np[idx]):.2e}, Entropy: {numpy_entropies[idx]:.2f}')

# Plot the maximum scar entropy in yellow and add to handles and labels
max_scar_handle, = ax.plot(np.mean(red_eigenvalues), rnd_scar_entropy, 'x', color='yellow', markersize=7)
handles.append(max_scar_handle)
labels.append('Max Scar, Entropy: {:.2f}'.format(rnd_scar_entropy))

# Sort handles and labels by lambda min
sorted_handles_labels = sorted(zip(handles, labels), key=lambda hl: float(hl[1].split(': ')[1].split(',')[0]) if 'λ_min' in hl[1] else float('inf'))
sorted_handles, sorted_labels = zip(*sorted_handles_labels)

# Filter min_eigenvalues to include only those corresponding to red_indices
filtered_min_eigenvalues = [min_eigenvalues_np[idx] for idx in red_indices_np]

# Create a color bar for the red points using filtered min_eigenvalues
norm = Normalize(vmin=min(filtered_min_eigenvalues), vmax=max(filtered_min_eigenvalues))
sm = ScalarMappable(cmap='rainbow', norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, label=r'$\lambda_{\mathrm{min}}$')

ax.set_xlabel('Eigenvalue of H')
ax.set_ylabel('Entanglement entropy')
#ax.set_title('Entanglement entropy as a function of energy eigenvalues')
plt.show()

In [ ]:
#################### OPTIMAL LINEAR COMBINATION OF SCARS THAT  MINIMIZE/MAXIMIZE EE ##########################

In [ ]:
# Let's look closer at the scar points with the lowest and highest entropy (Indices: 1267, 1268)

# MIN ENTROPY

lowest_entropy_index = 1267
lowest_entropy_eigenvector = eigenvectors[:, lowest_entropy_index]
entanglement_entropy_value = entanglement_entropy(lowest_entropy_eigenvector, half, N)
print(f'Lowest Entropy Index: {lowest_entropy_index}, Entropy: {entanglement_entropy_value}')   

# Convert the eigenvector to PyTorch tensors for real and imaginary parts
params_real = torch.tensor(lowest_entropy_eigenvector.real, dtype=torch.float, requires_grad=True)
params_imag = torch.tensor(lowest_entropy_eigenvector.imag, dtype=torch.float, requires_grad=True)

print("Initial coeffs, real:", params_real)
print("Initial coeffs, imag:", params_imag)

entanglement_entropy_value = entanglement_entropy_torch(lowest_entropy_eigenvector, half, N)
print(f'Lowest Entropy Index: {lowest_entropy_index}, Entropy: {entanglement_entropy_value}')

# Define coefficients for the linear combination
min_coeffs = np.zeros(len(red_eigenvectors_np), dtype=np.complex128)  # Initialize all coefficients to 0
min_coeffs[red_indices_np.index(1267)] = 1  # Ensure the coefficient for index 1267 is explicitly set to 1


# MAX ENTROPY
highest_entropy_index = 1268
highest_entropy_eigenvector = eigenvectors[:, highest_entropy_index]
entanglement_entropy_value = entanglement_entropy(highest_entropy_eigenvector, half, N)
print(f'Highest Entropy Index: {highest_entropy_index}, Entropy: {entanglement_entropy_value}') 

# Convert the eigenvector to PyTorch tensors for real and imaginary parts
params_real = torch.tensor(lowest_entropy_eigenvector.real, dtype=torch.float, requires_grad=True)
params_imag = torch.tensor(lowest_entropy_eigenvector.imag, dtype=torch.float, requires_grad=True)

print("Initial coeffs, real:", params_real)
print("Initial coeffs, imag:", params_imag)

entanglement_entropy_value = entanglement_entropy_torch(highest_entropy_eigenvector, half, N)
print(f'Highest Entropy Index: {highest_entropy_index}, Entropy: {entanglement_entropy_value}')

# Define coefficients for the linear combination
max_coeffs = np.zeros(len(red_eigenvectors_np), dtype=np.complex128)  # Initialize all coefficients to 0
max_coeffs[red_indices_np.index(1268)] = 1  # Ensure the coefficient for index 1268 is explicitly set to 1


In [ ]:
# find entropy min coeffs - everything  has to be in torch

# Initialize parameters for the coefficients with all values equal
#params_real = torch.full((5,), 1/np.sqrt(10), requires_grad=True)
#params_imag = torch.full((5,), 1/np.sqrt(10), requires_grad=True)

# Initialize parameters for the coefficients with random values
#params_real = torch.randn(5, requires_grad=True)
#params_imag = torch.randn(5, requires_grad=True)

# Convert random_coeffs to PyTorch tensors
#params_real = torch.tensor(random_coeffs.real, dtype=torch.float, requires_grad=True)
#params_imag = torch.tensor(random_coeffs.imag, dtype=torch.float, requires_grad=True)

red_eigenvectors = [torch.tensor(vec, dtype=torch.complex64) for vec in red_eigenvectors_np]


params_real = torch.tensor(min_coeffs.real, dtype=torch.float, requires_grad=True)
params_imag = torch.tensor(min_coeffs.imag, dtype=torch.float, requires_grad=True)

# Compute the linear combination using the defined function
psi = linear_combination(torch.complex(params_real, params_imag), red_eigenvectors)

# if the outer products between scarred states are computed before optimization loop
# ....

# Compute entanglement entropy
entropy = entanglement_entropy_torch(psi, half, N) 
print(f'Initial Entropy: {entropy.item()}')

# Normalize the parameters to ensure the norm is 1
with torch.no_grad():
    norm = torch.norm(torch.complex(params_real, params_imag))
    params_real /= norm
    params_imag /= norm

# Define the optimizer
optimizer = optim.Adam([params_real, params_imag], lr=0.01)

# Lists to store the values of parameters and entropy
coeffs_history_min = []
entropy_history = []

# Optimization loop
num_iterations = 150
for _ in range(num_iterations):
    optimizer.zero_grad()
    
    # Combine real and imaginary parts for the coefficients
    coeffs = torch.complex(params_real, params_imag)
    
    # Normalize coefficients to project them onto the unit sphere
    coeffs = coeffs / torch.norm(coeffs)
    
    # Compute psi
    psi = linear_combination(coeffs, red_eigenvectors)
    
    # Compute entanglement entropy
    entropy = entanglement_entropy_torch(psi, half, N) # it would be better to compute the outer product of the state before the optimization loop. but optimization is not taking too long so it's ok for now
    
    # Backpropagation
    entropy.backward()
    optimizer.step()
    
    # Normalize parameters again to ensure they stay on the unit sphere
    with torch.no_grad():
        norm = torch.norm(torch.complex(params_real, params_imag))
        params_real /= norm
        params_imag /= norm
    print("Coeffs norm:", torch.norm(torch.complex(params_real, params_imag)).detach().cpu().numpy())
    
    print(f'Entropy: {entropy.item()}')
    print(f'Coeffs: {coeffs.detach().cpu().numpy()}')

    # Store the values of parameters and entropy
    coeffs_history_min.append(coeffs.detach().cpu().numpy())
    entropy_history.append(entropy.item())

# Final optimized coefficients
optimized_coeffs = torch.complex(params_real, params_imag)
optimized_coeffs = optimized_coeffs / torch.norm(optimized_coeffs)

print("Optimized coefficients:", optimized_coeffs)

# Plot the value of parameters and entropy as a function of num_iterations
plt.figure(figsize=(12, 6))

# Plot entropy
plt.subplot(1, 2, 1)
plt.plot(entropy_history)
plt.xlabel('Iteration')
plt.ylabel('Entropy')
#plt.title('Entropy vs. Iterations')

# Plot coefficients
plt.subplot(1, 2, 2)
coeffs_history_min = np.array(coeffs_history_min)
for i in range(coeffs_history_min.shape[1]):
    plt.plot(coeffs_history_min[:, i].real, label=f'Coeff {i+1} Real')
    plt.plot(coeffs_history_min[:, i].imag, label=f'Coeff {i+1} Imag', linestyle='--')
plt.xlabel('Iteration')
plt.ylabel('Coefficient Value')
#plt.title('Coefficients vs. Iterations')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Compute the max_scar wavefunction using the optimized coefficients (min entropy)
min_scar = linear_combination_np(coeffs_history_min[-1], red_eigenvectors_np)
# Define a list of colors for red points
red_colors = plt.cm.rainbow(np.linspace(0, 1, len(red_indices_np)))

# Compute entanglement entropy for the max_scar wavefunction
min_scar_entropy = entanglement_entropy(min_scar, half, N)

# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices_np:
    print(f'Index: {i}, Entropy: {numpy_entropies[i]}')

print(f'Min scar entropy: {min_scar_entropy}')

# Plot entanglement entropy as a function of energy eigenvalues
fig, ax = plt.subplots(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_np]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    ax.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=2)

# Collect handles and labels for the legend
handles = []
labels = []

# Plot each red point with a different color and collect handles and labels
for i, idx in enumerate(red_indices_np):
    handle, = ax.plot(eigenvalues[idx], numpy_entropies[idx], 'o', color=red_colors[i], markersize=5)
    handles.append(handle)
    labels.append(r'$\lambda_\mathrm{min}$: {float(min_eigenvalues[idx]):.2e}, Entropy: {naive_entropies[idx]:.2f}')

# Plot the maximum scar entropy in yellow and add to handles and labels
min_scar_handle, = ax.plot(np.mean(eigenvalues[red_indices_np]), min_scar_entropy, 'x', color='black', markersize=7)
handles.append(min_scar_handle)
labels.append('Min Scar, Entropy: {:.2f}'.format(min_scar_entropy))

# Sort handles and labels by lambda min
sorted_handles_labels = sorted(zip(handles, labels), key=lambda hl: float(hl[1].split(': ')[1].split(',')[0]) if 'λ_min' in hl[1] else float('inf'))
sorted_handles, sorted_labels = zip(*sorted_handles_labels)

# Filter min_eigenvalues to include only those corresponding to red_indices
filtered_min_eigenvalues = [min_eigenvalues_np[idx] for idx in red_indices_np]

# Create a color bar for the red points using filtered min_eigenvalues
norm = Normalize(vmin=min(filtered_min_eigenvalues), vmax=max(filtered_min_eigenvalues))
sm = ScalarMappable(cmap='rainbow', norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, label=r'$\lambda_{\mathrm{min}}$')

ax.set_xlabel('Eigenvalue of H')
ax.set_ylabel('Entanglement entropy')
plt.show()

In [ ]:
# now find entropy max coeffs - everything  has to be in torch

# Initialize parameters for the coefficients with all values equal
#params_real = torch.full((5,), 1/np.sqrt(10), requires_grad=True)
#params_imag = torch.full((5,), 1/np.sqrt(10), requires_grad=True)

# Initialize parameters for the coefficients with random values
#params_real = torch.randn(5, requires_grad=True)
#params_imag = torch.randn(5, requires_grad=True)

# Convert random_coeffs to PyTorch tensors
#params_real = torch.tensor(random_coeffs.real, dtype=torch.float, requires_grad=True)
#params_imag = torch.tensor(random_coeffs.imag, dtype=torch.float, requires_grad=True)

params_real = torch.tensor(max_coeffs.real, dtype=torch.float, requires_grad=True)
params_imag = torch.tensor(max_coeffs.imag, dtype=torch.float, requires_grad=True)

print("Initial coeffs, real", params_real)
print("Initial coeffs imag",  params_imag)

# Compute psi
psi = linear_combination(torch.complex(params_real, params_imag), red_eigenvectors)

# Compute entanglement entropy
entropy = entanglement_entropy_torch(psi, half, N) 
print(f'Initial Entropy: {entropy.item()}')

# Normalize the parameters to ensure the norm is 1
with torch.no_grad():
    norm = torch.norm(torch.complex(params_real, params_imag))
    params_real /= norm
    params_imag /= norm

# Define the optimizer
optimizer = optim.Adam([params_real, params_imag], lr=0.01)

# Lists to store the values of parameters and entropy
coeffs_history_max = []
entropy_history_max = []

# Optimization loop
num_iterations = 150
for _ in range(num_iterations):
    optimizer.zero_grad()
    
    # Combine real and imaginary parts for the coefficients
    coeffs = torch.complex(params_real, params_imag)
    
    # Normalize coefficients to project them onto the unit sphere
    coeffs = coeffs / torch.norm(coeffs)
    
    # Compute psi
    psi = linear_combination(coeffs, red_eigenvectors)
    
    # Compute - entanglement entropy
    entropy = - entanglement_entropy_torch(psi, half, N) # it would be better to compute the outer product of the state before the optimization loop. but optimization is not taking too long so it's ok for now
    
    # Backpropagation
    entropy.backward()
    optimizer.step()
    
    # Normalize parameters again to ensure they stay on the unit sphere
    with torch.no_grad():
        norm = torch.norm(torch.complex(params_real, params_imag))
        params_real /= norm
        params_imag /= norm
    print("Coeffs norm:", torch.norm(torch.complex(params_real, params_imag)).detach().cpu().numpy())
    
    print(f'Entropy: {abs(entropy.item())}')
    print(f'Coeffs: {coeffs.detach().cpu().numpy()}')

    # Store the values of parameters and entropy
    coeffs_history_max.append(coeffs.detach().cpu().numpy())
    entropy_history_max.append(abs(entropy.item()))

# Final optimized coefficients
optimized_coeffs_max = torch.complex(params_real, params_imag)
optimized_coeffs_max = optimized_coeffs_max / torch.norm(optimized_coeffs_max)

print("Optimized coefficients:", optimized_coeffs_max)

# Plot the value of parameters and entropy as a function of num_iterations
plt.figure(figsize=(12, 6))

# Plot entropy
plt.subplot(1, 2, 1)
plt.plot(np.abs(entropy_history_max))
plt.xlabel('Iteration')
plt.ylabel('Entropy')
#plt.title('Entropy vs. Iterations')

# Plot coefficients
plt.subplot(1, 2, 2)
coeffs_history_max = np.array(coeffs_history_max)
for i in range(coeffs_history_max.shape[1]):
    plt.plot(coeffs_history_max[:, i].real, label=f'Coeff {i+1} Real')
    plt.plot(coeffs_history_max[:, i].imag, label=f'Coeff {i+1} Imag', linestyle='--')
plt.xlabel('Iteration')
plt.ylabel('Coefficient Value')
#plt.title('Coefficients vs. Iterations')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Compute the max_scar wavefunction using the optimized coefficients (min entropy)
max_scar = linear_combination_np(coeffs_history_max[-1], red_eigenvectors_np)
# Define a list of colors for red points
red_colors = plt.cm.rainbow(np.linspace(0, 1, len(red_indices_np)))

# Compute entanglement entropy for the max_scar wavefunction
max_scar_entropy = entanglement_entropy(max_scar, half, N)

# Print entropies of red points only
print("Entropies of red points:")
for i in red_indices_np:
    print(f'Index: {i}, Entropy: {numpy_entropies[i]}')

print(f'Max scar entropy: {max_scar_entropy}')

# Plot entanglement entropy as a function of energy eigenvalues
fig, ax = plt.subplots(figsize=(10, 6))

# Highlight points with y-component less than the threshold in red
colors = ['red' if y < threshold else 'C0' for y in min_eigenvalues_np]

# Plot entanglement entropy with highlighted points
for i, color in enumerate(colors):
    ax.plot(eigenvalues[i], numpy_entropies[i], 'o', color=color, markersize=2)

# Collect handles and labels for the legend
handles = []
labels = []

# Plot each red point with a different color and collect handles and labels
for i, idx in enumerate(red_indices_np):
    handle, = ax.plot(eigenvalues[idx], numpy_entropies[idx], 'o', color=red_colors[i], markersize=5)
    handles.append(handle)
    labels.append(r'$\lambda_\mathrm{min}$: {float(min_eigenvalues[idx]):.2e}, Entropy: {naive_entropies[idx]:.2f}')

# Plot the maximum scar entropy in yellow and add to handles and labels
max_scar_handle, = ax.plot(np.mean(eigenvalues[red_indices_np]), max_scar_entropy, 'x', color='black', markersize=7)
handles.append(max_scar_handle)
labels.append('Max Scar, Entropy: {:.2f}'.format(max_scar_entropy))

# Sort handles and labels by lambda min
sorted_handles_labels = sorted(zip(handles, labels), key=lambda hl: float(hl[1].split(': ')[1].split(',')[0]) if 'λ_min' in hl[1] else float('inf'))
sorted_handles, sorted_labels = zip(*sorted_handles_labels)

# Filter min_eigenvalues to include only those corresponding to red_indices
filtered_min_eigenvalues = [min_eigenvalues_np[idx] for idx in red_indices_np]

# Create a color bar for the red points using filtered min_eigenvalues
norm = Normalize(vmin=min(filtered_min_eigenvalues), vmax=max(filtered_min_eigenvalues))
sm = ScalarMappable(cmap='rainbow', norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, label=r'$\lambda_{\mathrm{min}}$')

ax.set_xlabel('Eigenvalue of H')
ax.set_ylabel('Entanglement entropy')
#ax.set_title('Entanglement entropy as a function of energy eigenvalues')
#ax.legend(sorted_handles, sorted_labels)
plt.show()

In [ ]:
################## PROPERTIES OF EACH SCARRED STATE AND THEIR RDM ######################

In [ ]:
# List to store the number of dependent columns and their indices for each matrix
dependent_columns_info = []

for idx, rdm in enumerate(min_rdms_np):
    print(f"RDM Index: {idx}")
    
    # Perform QR decomposition with column pivoting
    Q, R, pivot_indices = qr(rdm, pivoting=True)
    
    # Determine rank using a threshold on the diagonal of R
    tol = 1e-12
    rank = np.sum(np.abs(np.diag(R)) > tol)
    
    # Calculate the number of dependent columns
    num_dependent_columns = 16 - rank
    
    # Identify dependent columns
    dependent_columns = sorted(set(range(16)) - set(pivot_indices[:rank]))
    dependent_columns_info.append((num_dependent_columns, dependent_columns))
    
    print(f"Number of dependent columns: {num_dependent_columns}")
    print(f"Dependent columns indices: {dependent_columns}")
    print("-" * 50)

# Print the results
print("Dependent columns info for each matrix:", dependent_columns_info)

In [ ]:
# List to store the null space information for each matrix
null_spaces_info = []

for idx, rdm in enumerate(min_rdms_np):
    print(f"RDM Index: {idx}")
    
    # Compute the null space of the RDM
    null_space_rdm = null_space(rdm)
    
    # Store the null space information
    null_spaces_info.append(null_space_rdm)
    
    # Print the null space dimensions and basis vectors
    print(f"Null space dimension: {null_space_rdm.shape[1]}")
    print(f"Null space basis vectors:\n{null_space_rdm}")
    print("-" * 50)

    # Apply the RDM to each null basis vector
    for i, basis_vector in enumerate(null_space_rdm.T):  # Transpose to iterate over columns
        result = np.dot(rdm, basis_vector)
        print(f"RDM {idx}, Null Basis Vector {i}:")
        print(f"Result: {result}")
        print(f"Norm of Result: {np.linalg.norm(result)}")

# Print the results
print("Null space information for each matrix computed.")

In [ ]:
# For each RDM, check if any row in its null space basis matrix is (close to) zero
for idx, rdm in enumerate(min_rdms_np):
    null_space_rdm = null_space(rdm)
    for i, row in enumerate(null_space_rdm):
        if np.allclose(row, 0, atol=1e-12):
            print(f"RDM {idx}: Row {i} in null space is (close to) zero.")

In [ ]:
# Count the number of distinct rows in the null space basis of RDM 0
rdm = min_rdms_np[4]
null_space_rdm = null_space(rdm)

# Use np.allclose to group rows that are numerically identical
distinct_rows = []
for i, row in enumerate(null_space_rdm):
    is_new = True
    for drow in distinct_rows:
        if np.allclose(row, drow, atol=1e-12):
            is_new = False
            break
    if is_new:
        distinct_rows.append(row)

print(f"Number of distinct rows in null space basis of RDM 0: {len(distinct_rows)}")

In [ ]:
# Print the binary basis states for each group of identical rows in the null space basis of RDM 0 (5 spins)
# Also print the smallest row index for each group

def index_to_binary(index, num_qubits):
    return format(index, f'0{num_qubits}b')

rdm = min_rdms_np[0]
null_space_rdm = null_space(rdm)

# Group row indices by their unique row (up to numerical tolerance)
groups = []
group_indices = []

for i, row in enumerate(null_space_rdm):
    found = False
    for g, grow in enumerate(groups):
        if np.allclose(row, grow, atol=1e-12):
            group_indices[g].append(i)
            found = True
            break
    if not found:
        groups.append(row)
        group_indices.append([i])

print(f"Number of distinct rows: {len(groups)}")
for indices in group_indices:
    basis_states = [index_to_binary(idx, 5) for idx in indices]
    min_row = min(indices)
    print(f"Row {min_row}: Basis states: {', '.join(basis_states)}")

In [ ]:
# List to store the null space information for each matrix
null_spaces_info = []

for idx, rdm in enumerate(min_rdms_np):
    print(f"RDM Index: {idx}")
    
    # Compute the null space of the RDM
    null_space_rdm = null_space(rdm)
    
    # Store the null space information
    null_spaces_info.append(null_space_rdm)
    
    # Print the null space dimensions and basis vectors
    print(f"Null space dimension: {null_space_rdm.shape[1]}")
    #print(f"Null space basis vectors:\n{null_space_rdm}")
    print("-" * 50)

    # Apply the RDM to each null basis vector
    #for i, basis_vector in enumerate(null_space_rdm.T):  # Transpose to iterate over columns
        #result = np.dot(rdm, basis_vector)
        #print(f"RDM {idx}, Null Basis Vector {i}:")
        #print(f"Result: {result}")
        #print(f"Norm of Result: {np.linalg.norm(result)}")
    
    # Check for repeating rows in the null space basis vectors
    print(f"Repeating rows in the null space basis vectors for RDM {idx}:")
    for i in range(null_space_rdm.shape[0]):
        for j in range(i + 1, null_space_rdm.shape[0]):
            if np.allclose(null_space_rdm[i, :], null_space_rdm[j, :], atol=1e-12):  # Compare rows with a tolerance
                print(f"Row {i} is identical to Row {j}")
    print("-" * 50)

In [ ]:
# Iterate over scarred eigenvectors
for i, eigenvector in enumerate(red_eigenvectors_np):
    # Print the entire eigenvector
    non_zero_count = 0  # Counter for non-zero components
    print(f"Full Eigenvector {i}:")
    for index, component in enumerate(eigenvector):
        # Set components of the order 10^-12 or smaller to zero
        if np.abs(component) < 1e-12: # I noticed that  most components are of the order 10^-13 or smaller - only a handful are mucgh larger
            eigenvector[index] = 0.0
        else:
            non_zero_count += 1
        binary_basis = index_to_binary(index, N)
        print(f"{binary_basis}: {eigenvector[index]}")

    print(f"Total Non-Zero Components in Scarred Eigenvector {i}: {non_zero_count}")
    print("=" * 50)

In [ ]:
# Iterate over scarred eigenvectors
for i, eigenvector in enumerate(red_eigenvectors_np):
    print(f"Scarred Eigenvector {i}:")
    
    # Dictionary to track processed coefficients
    processed_coeffs = set()
    
    # Iterate over components of the eigenvector
    for index, component in enumerate(eigenvector):
        if np.abs(component) > 1e-12:  # Check if the component is non-zero
            binary_basis = index_to_binary(index, N)
            coeff = component
            
            # Skip if this coefficient (or its negative) has already been processed
            if coeff in processed_coeffs or -coeff in processed_coeffs:
                continue
            
            # Count the number of 1's in the binary representation
            num_ones = binary_basis.count('1')
            
            # Print the coefficient, binary representation, and number of 1's
            print(f"  Coefficient: {coeff}")
            print(f"    Binary Representation: {binary_basis}")
            print(f"    Elements [0, 4, 5, 9]: {binary_basis[0]}, {binary_basis[4]}, {binary_basis[5]}, {binary_basis[9]}")
            print(f"    Number of 1's: {num_ones}")
            
            # Check for the negative of the coefficient
            for j, other_component in enumerate(eigenvector):
                if np.abs(other_component + coeff) < 1e-10:  # Compare with tolerance
                    binary_basis_neg = index_to_binary(j, N)
                    num_ones_neg = binary_basis_neg.count('1')
                    print(f"  Negative Coefficient: {-coeff}")
                    print(f"    Binary Representation: {binary_basis_neg}")
                    print(f"    Elements [0, 4, 5, 9]: {binary_basis_neg[0]}, {binary_basis_neg[4]}, {binary_basis_neg[5]}, {binary_basis_neg[9]}")
                    print(f"    Number of 1's: {num_ones_neg}")
                    break
            
            # Mark this coefficient and its negative as processed
            processed_coeffs.add(coeff)
            processed_coeffs.add(-coeff)
    
    print("=" * 50)

In [ ]:
# List to store even swaps numbers
even_swaps_numbers = []

# Dictionary to track spin exchange arrays and their counts
spin_exchange_counts = {}

# Iterate over scarred eigenvectors
for i, eigenvector in enumerate(red_eigenvectors_np):
    print(f"Scarred Eigenvector {i}:")

    # Dictionary to track processed coefficients
    processed_coeffs = set()

    # Iterate over components of the eigenvector
    for index, component in enumerate(eigenvector):
        if np.abs(component) > 1e-12:  # Check if the component is non-zero
            binary_basis = index_to_binary(index, N)
            coeff = component

            # Skip if this coefficient (or its negative) has already been processed
            if coeff in processed_coeffs or -coeff in processed_coeffs:
                continue

            # Check for the negative of the coefficient
            for j, other_component in enumerate(eigenvector):
                if np.abs(other_component + coeff) < 1e-10:  # Compare with tolerance
                    binary_basis_neg = index_to_binary(j, N)

                    # Identify the spins that are exchanged
                    spin_exchange = [
                        k for k in range(N) if binary_basis[k] != binary_basis_neg[k]
                    ]

                    # Compute swaps number
                    swaps_number = len(spin_exchange) // 2

                    # Check if swaps number is odd
                    is_odd = swaps_number % 2 == 1

                    print(f"  Coefficient: {coeff}")
                    print(f"    Binary Representation: {binary_basis}")
                    print(f"  Negative Coefficient: {-coeff}")
                    print(f"    Binary Representation: {binary_basis_neg}")
                    print(f"    Spin Exchange: {spin_exchange}")
                    print(f"    Swaps Number: {swaps_number} (Odd: {is_odd})")

                    # Add to even swaps numbers if swaps number is even
                    if not is_odd:
                        even_swaps_numbers.append(swaps_number)

                    # Track spin exchange arrays
                    spin_exchange_tuple = tuple(spin_exchange)
                    if spin_exchange_tuple in spin_exchange_counts:
                        spin_exchange_counts[spin_exchange_tuple] += 1
                    else:
                        spin_exchange_counts[spin_exchange_tuple] = 1

                    break

            # Mark this coefficient and its negative as processed
            processed_coeffs.add(coeff)
            processed_coeffs.add(-coeff)

    print("=" * 50)

# Print the list of even swaps numbers
print("Even swaps numbers:", even_swaps_numbers)

# Check for repeated spin exchange arrays
repeated_spin_exchanges = [
    spin_exchange for spin_exchange, count in spin_exchange_counts.items() if count > 1
]
if repeated_spin_exchanges:
    print("Repeated spin exchange arrays:", repeated_spin_exchanges)
else:
    print("No repeated spin exchange arrays found.")

print(len(spin_exchange_counts), "spin exchange arrays found.")

In [ ]:
# min_scar

non_zero_count = 0  # Counter for non-zero components

for index, component in enumerate(min_scar):
    # Set components of the order 10^-12 or smaller to zero
    if np.abs(component) < 1e-12: # I noticed that  most components are of the order 10^-13 or smaller - only a handful are mucgh larger
        min_scar[index] = 0.0
    else:
        non_zero_count += 1
    binary_basis = index_to_binary(index, N)
    print(f"{binary_basis}: {np.real(min_scar[index])}")

print(f"Total Non-Zero Components in min_scar: {non_zero_count}")
print("=" * 50)

In [ ]:
# check if other states have 280 non-zero components - it's only the 5 scars!

# Count distinct non_zero_count values
distinct_counts = set()

# Counter to track occurrences of each non_zero_count
non_zero_count_occurrences = Counter()

# Iterate over all eigenvectors
for i in range(eigenvectors.shape[1]):
    eigenvector = eigenvectors[:, i]
    non_zero_count = 0  # Counter for non-zero components
    
    # Count non-zero components
    for component in eigenvector:
        if np.abs(component) >= 1e-12:
            non_zero_count += 1
    
    # Update the counter
    non_zero_count_occurrences[non_zero_count] += 1
    print(f"Total Non-Zero Components in Eigenvector {i}: {non_zero_count}")

# Print the number of distinct non_zero_count values
print(f"Number of distinct non_zero_count values: {len(non_zero_count_occurrences)}")
print(f"Distinct non_zero_count values: {sorted(non_zero_count_occurrences.keys())}")

# Print how many times each distinct non_zero_count appears
print("Occurrences of each non_zero_count:")
for count, occurrences in sorted(non_zero_count_occurrences.items()):
    print(f"Non-Zero Count: {count}, Occurrences: {occurrences}")

In [ ]:
'''#find a 4/5 spins charge commuting with the rdm - let's first use quantum channel to derive the effective action of the global C5 symmetry operator on the 4 spins subsystem:

##   rho4' = Tr_8[C5 (rho4 cross rho8) C5^dagger], and compare it to rho4
##   C5^4 = Tr_8[C5 (I4 cross rho8)] --- not sure if this is correct, but let's try it anyway

# let's first find rho8 - we already have rho4 and C5=P

###### USELESS ############

# Compute RDM on the complement of keep_qubits for scarred eigenstates
complement_rdms = []  # List to store RDMs on the complement of keep_qubits

# Define the complement of keep_qubits
all_qubits = set(range(N))
for i in red_indices_np:
    min_eigenvector = eigenvectors[:, i]
    density_matrix = np.outer(min_eigenvector, min_eigenvector.conj())
    density_matrix_qobj = Qobj(density_matrix, dims=[[2]*N, [2]*N])

    # Complement of keep_qubits
    complement_qubits = list(all_qubits - set(keep_qubits))

    # Trace out the complement qubits
    traced_out_complement = ptrace(density_matrix_qobj, complement_qubits)

    # Convert the result back to a dense matrix if needed
    traced_out_complement_dense = traced_out_complement.full()
    complement_rdms.append(traced_out_complement_dense)

    # Compute the trace of the complement RDM
    trace_value = np.trace(traced_out_complement_dense)

    # Check if the complement RDM is Hermitian
    is_hermitian = np.allclose(traced_out_complement_dense, traced_out_complement_dense.conj().T)

    # Print the results for debugging
    print(f"Complement RDM for eigenstate {i}:")
    print(f"Shape: {traced_out_complement_dense.shape}")
    print(f"Trace: {trace_value}")
    print(f"Is Hermitian: {is_hermitian}")
    print("-" * 50)

# Now `complement_rdms` contains the RDMs on the complement of keep_qubits for all scarred eigenstates

# full P=C5

P = dok_matrix((2**N, 2**N), dtype=np.float64)

# Go through all basis states
for index in range(2**N):
    # Convert index to spin bits
    state_bits = list(map(int, np.binary_repr(index, width=N)))
    
    # Apply the permutation
    new_bits = apply_permutation(state_bits, N, perm)
    
    # Convert back to integer index
    new_index = int("".join(map(str, new_bits)), 2)
    
    # Set the permutation matrix entry
    P[new_index, index] = 1.0

# Optionally convert to CSR format for efficient multiplication
P = P.toarray()

print("Permutation matrix P constructed with shape:", P.shape)

# first attempt for the effective 4-spins operator C5^4

C5_4s = []

P_qobj = Qobj(P, dims=[[2]*N, [2]*N])


for i in range(len(complement_rdms)):
    rho8 = complement_rdms[i]
    # Convert rho8 (NumPy matrix) back to Qobj
    rho8_qobj = Qobj(rho8, dims=[[2] * (N - len(keep_qubits)), [2] * (N - len(keep_qubits))])

    # Define subsystem dimensions
    dim_A = 2**(len(keep_qubits))   # e.g., 4 spins → dimension 16
    dim_B = rho8_qobj.dims[0][0]  # This should be 2**(N-4) for N spins total

    # Identity on subsystem A
    IA_qobj = qeye(dim_A)
    IA = np.eye(dim_A, dtype=np.float64)  # Identity matrix for subsystem A

    # Build the product state (I_A ⊗ rhoB)
    #IA_rho8 = tensor(IA, rho8_qobj)
    IA_rho8 = np.kron(IA, rho8)  # Kronecker product of I_A and rhoB
    IA_rho8_qobj = Qobj(IA_rho8, dims=[[2] * N, [2] * N])

    # Print dimensions
    print(f"Dimension of IA: {IA.shape}")
    print(f"Dimension of rho8_qobj: {rho8.shape}")
    print(f"Dimension of IA_rho8: {IA_rho8_qobj.dims}")

    # Apply global C5 (P) to this state
    U_rho = P_qobj * IA_rho8_qobj

    # Partial trace over subsystem B → result acts on subsystem A
    # If A is the first subsystem in the tensor product, trace out B (which is subsystem 1)
    #C5_4 = ptrace(U_rho, 0)  # 0 → subsystem A (remaining after tracing out B)
    C5_4 = ptrace(U_rho, keep_qubits)  # Trace out the complement of keep_qubits

    # Now C5_tilde is the effective operator acting on subsystem A (4 spins)
    C5_4s.append(C5_4.full())  # Convert to dense matrix if needed
    print(f"Effective operator C5^4 for eigenstate {i}:")
    print(f"Shape: {C5_4.shape}")

for i in range(len(C5_4s)):
    print(f"Effective operator C5^4 for eigenstate {i}:")
    print(f"Shape: {C5_4s[i].shape}")
    print(C5_4s[i])
    print("-" * 50)

# Compute commutators between each C5_4 and the corresponding rho4
commutators = []

for i in range(len(C5_4s)):
    # Convert rho4 (min_rdms) to a dense matrix if needed
    rho4 = min_rdms_np[i]  # Corresponding reduced density matrix (rho4)
    C5_4 = C5_4s[i]     # Corresponding effective operator (C5_4)

    # Compute the commutator: [C5_4, rho4] = C5_4 * rho4 - rho4 * C5_4
    commutator = np.dot(C5_4, rho4) - np.dot(rho4, C5_4)
    commutators.append(commutator)

    # Print the commutator for debugging
    print(f"Commutator for scar {i}:")
    print(commutator)
    print(f"Norm of commutator: {np.linalg.norm(commutator)}")
    print("-" * 50)

# Now `commutators` contains the commutators for all eigenstates

rho4Us = []

for i in range(len(complement_rdms)):

    rho4 = min_rdms_np[i]
    # Convert rho4 (NumPy matrix) back to Qobj
    rho4_qobj = Qobj(rho4, dims=[[2] * (len(keep_qubits)), [2] * (len(keep_qubits))])

    rho8 = complement_rdms[i]
    # Convert rho8 (NumPy matrix) back to Qobj
    rho8_qobj = Qobj(rho8, dims=[[2] * (N - len(keep_qubits)), [2] * (N - len(keep_qubits))])

    # Build the product state (rhoA ⊗ rhoB)
    rho4_rho8 = np.kron(rho4, rho8)  # Kronecker product of rhoA and rhoB
    rho4_rho8_qobj = Qobj(rho4_rho8, dims=[[2] * N, [2] * N])

    # Apply C5 and its adjoint to this state
    U_rho_U = P_qobj * rho4_rho8_qobj * (P_qobj).dag()

    # Partial trace over subsystem B → result acts on subsystem A
    rho4U = ptrace(U_rho_U, keep_qubits)  # Trace out the complement of keep_qubits

    # rho4U is the 4 spins rdm after effective action of C5 on subsystem A (4 spins)
    rho4U_dense = rho4U.full()  # Convert to dense matrix if needed
    rho4Us.append(rho4U_dense)

    # Check trace
    trace_value = np.trace(rho4U_dense)
    print(f"Trace of rho4U for eigenstate {i}: {trace_value}")

    # Check if Hermitian
    is_hermitian = np.allclose(rho4U_dense, rho4U_dense.conj().T)
    print(f"Is rho4U Hermitian for eigenstate {i}: {is_hermitian}")

    print(f"rho4U for eigenstate {i}:")
    print(f"Shape: {rho4U_dense.shape}")
    print("-" * 50)


for i in range(len(rho4Us)):
    norm_rho4U = np.linalg.norm(rho4Us[i], ord='fro')  # Frobenius norm
    print(f"Norm of rho4U for eigenstate {i}: {norm_rho4U}")
    norm_rho4 = np.linalg.norm(min_rdms_np[i], ord='fro')  # Frobenius norm
    print(f"Norm of rho4 for eigenstate {i}: {norm_rho4}")
    distance = np.linalg.norm(rho4Us[i] - min_rdms_np[i], ord='fro')  # Frobenius norm
    print(f"Distance between rho4U and rho4 for eigenstate {i}: {distance}")'''